In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min
from scipy.optimize import brentq

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
import math
import pickle

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div, call_delta_with_spot_and_div, call_delta_with_spot, put_delta_with_spot, put_delta_with_spot_and_div 

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date



In [2]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "ATM"

#Product type to hedge: either "CE" or "PE"
prod_type = "CE"
# prod_type = "PE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path
out_PnL_path = source_path + "doc_outputs/PnL_attribution/"


# Periods of interest will be a dictionar/y
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
# periods_of_interest = {2020: [4, 5]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1


# Option Greeks
Normal CDF function is $\Phi(.)$ and pdf function is $\phi(.)$ <br>
$r_f$ is risk free interest rate, $q$is dividend and $r=r_f-q$ <br>
 
Call Option Delta ($\frac{dV}{dS}$): $exp(-q.dt) * \Phi(d_1)$ <br>
Put Option Delta ($\frac{dV}{dS}$): $- exp(-q.dt) * \Phi(d_1) $ <br>
Vega (Call and Put) ($\frac{dV}{d\sigma}$): $S_t * exp(-q.dt) * \phi(d_1) * \sqrt{dt} = K * exp(-r_f.dt) * \phi(d_2) * \sqrt{dt}$ <br>
Theta (Call) ($\frac{dV}{dT}$): $- exp^{(-q.dt)}\frac{S_t \phi(d_1) \sigma}{2\sqrt{dt}} - r K exp(-r.dt) \Phi(d_2) + q S exp(-q.dt) \Phi(d_1)$ <br>
Theta (put) ($\frac{dV}{dT}$): $- exp(-q.dt)\frac{S_t \phi(d_1) \sigma}{2\sqrt{dt}} + r K exp(-r.dt) \Phi(-d_2) - q S exp(-q.dt) \Phi(-d_1)$ <br>
rho (call) ($\frac{dV}{dr}$): $Kdtexp(-rdt)\Phi(d_2)$ <br>
rho (put) ($\frac{dV}{dr}$): $-Kdtexp(-rdt)\Phi(-d_2)$ <br>
gamma (call and put) ($\frac{d^{2}V}{dS^{2}}$): $ exp^{(-qdt)} \frac{\phi(d_1)}{S \sigma \sqrt{dt}} = K exp^{(-rdt)} \frac{\phi(d_2)}{S^{2}\sigma\sqrt{dt}} = \frac{Vega}{S^{2} \sigma dt}$ <br>
Vanna or DVegaDspot or DdeltaDvol (call and put) ($\frac{d^{2}V}{dSd\sigma}$): $exp^{(-qdt)} \phi(d_1) \frac{d_2}{\sigma} = \frac{Vega}{S} * (1 -\frac{d_1}{\sigma \sqrt{dt}})$ <br>
volga or vomma  (call and put) ($\frac{d^{2}V}{d\sigma^{2}}$): $Sexp^{(-qdt)}\phi(d_1)\sqrt{dt} \frac{d_1 d_2}{\sigma} = Vega * \frac{d_1 d_2}{\sigma}$

In [3]:
#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

# def error_func(vol, mkt_price, stock_vec, opt_strike, r, r_f, dt):
#     error = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol, dt) - mkt_price
#     return error

# def error_func_short(iv, mkt_price, s_t, k, r, r_f, dt, option_type):
#     if (option_type == "CE"):
#         model_price = BSM_call_vec_with_div(s_t, k, r, r_f, iv, dt)
#     else:
#         model_price = BSM_put_vec_with_div(s_t, k, r, r_f, iv, dt)
#     error = model_price - mkt_price
#     return error

def error_func(iv, mkt_price, s_t, k, r, r_f, dt, option_type):
    if (option_type == "CE"):
        try:
            model_price = BSM_call_vec_with_div(s_t, k, r, r_f, iv, dt)
        except:
            model_price = None
    else:
        try:
            model_price = BSM_put_vec_with_div(s_t, k, r, r_f, iv, dt)
        except:
            model_price = None
    try:
        error = model_price - mkt_price
    except:
        error = None
    return error


def find_vega(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    return (norm.pdf(d_2) * k * np.exp(-r_f * dt)) * np.sqrt(dt)

# def find_vega_test(s_t, k, r, r_f, vol, dt):
#     d_1_c = (1 / (vol * (dt ** 0.5))) 
#     d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
#     d_1 = d_1_c * d1_log
#     d_2 = d_1 - vol * (dt ** 0.5)
#     q = r_f - r
#     return (norm.pdf(d_1) * s_t * np.exp(-q * dt)) * np.sqrt(dt)

def find_gamma(s_t, k, r, r_f, vol, dt):
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    gamma = vega / ((s_t**2) * vol * dt)
    return gamma

# def find_gamma_test(s_t, k, r, r_f, vol, dt):
#     d_1_c = (1 / (vol * (dt ** 0.5))) 
#     d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
#     d_1 = d_1_c * d1_log
#     d_2 = d_1 - vol * (dt ** 0.5)
#     q = r_f - r
#     gamma = np.exp(-q*dt) * (norm.pdf(d_1)) / (vol * s_t * np.sqrt(dt))
#     return gamma

def find_vanna(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    vanna = (vega / s_t) * (1 - ((d_1)/ (vol * np.sqrt(dt))))
    return vanna

def find_volga(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    volga = vega * (d_1 * d_2) / vol
    return volga

def find_speed(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    gamma = find_gamma(s_t, k, r, r_f, vol, dt)
    speed = - (gamma / s_t) * ((d_1 / (vol * np.sqrt(dt))) + 1)
    return speed


def find_theta_Call(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    term1 = - (np.exp(-q*dt) * s_t * norm.pdf(d_1) * vol) / (2 * np.sqrt(dt))
    term2 = - r_f * k * np.exp(-r_f*dt) * norm.cdf(d_2)
    term3 = q * s_t * np.exp(-q*dt) * norm.cdf(d_1)
    theta = term1 + term2 + term3
    return theta

def find_theta_Put(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    term1 = - (np.exp(-q*dt) * s_t * norm.pdf(d_1) * vol) / (2 * np.sqrt(dt))
    term2 =  r_f * k * np.exp(-r_f*dt) * norm.cdf(-d_2)
    term3 = -q * s_t * np.exp(-q*dt) * norm.cdf(-d_1)
    theta = term1 + term2 + term3
    return theta


def find_veta(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    veta = -s_t * np.exp(-q*dt) * norm.pdf(d_1) * np.sqrt(dt) * \
    (q + ((r_f-q) * d_1) / (vol * np.sqrt(dt)) - (1 + d_1 * d_2) / (2 * dt) )
    return veta

def find_charm_call(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    charm = q * np.exp(-q*dt) * norm.cdf(d_1) - \
            np.exp(-q*dt) * norm.pdf(d_1) * \
            (( 2* (r_f - q) * dt - d_2 * vol * np.sqrt(dt)) / (2 * dt * vol * np.sqrt(dt)))
    return charm
 
def find_charm_put(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    charm = - q * np.exp(-q*dt) * norm.cdf(-d_1) - \
            np.exp(-q*dt) * norm.pdf(d_1) * \
            (( 2* (r_f - q) * dt - d_2 * vol * np.sqrt(dt)) / (2 * dt * vol * np.sqrt(dt)))
    return charm


In [120]:
###########################################################
# Target Option PnL Attribution - 7th April - NIFTY CALL ##
###########################################################

prv_day = date(2020,4,3)
nxt_day = date(2020,4,7)
begin_wk = date(2020,4,1)
end_wk = date(2020,4,9)
end_mth = date(2020,4,30)
prev_price_S0 = 8083.8
next_price_S1 = 8792.2

summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
                "S0": [prev_price_S0 for i in range(0,3)], "S1": [next_price_S1 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
                 "K": [8600, 8000, 9500], "MV0": [230.25, 526.15, 44.4], "MV1": [590.25, 1045.55, 150.55], \
                  "dt0": [((end_mth - prv_day).days)/365 for i in range(0,3)], \
                  "dt1": [((end_mth - nxt_day).days)/365 for i in range(0,3)]}

#Incremental Analysis

#Long Term Option
df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_ds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_summary["BS0_dv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_dt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


# Greeks
df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

# df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )


#PnL
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["iv1"] - x["iv0"]) *(x["S1"] - x["S0"]), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * (x["dt0"] - x["dt1"]) , axis=1 )
df_summary["veta PnL"] = df_summary.apply(lambda x: x["veta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S1"] - x["S0"]),3)), axis=1 )
df_summary["charm PnL"] = df_summary.apply(lambda x: x["charm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )

                                         
df_summary["Target Option Realised PnL"] = df_summary["MV1"] - df_summary["MV0"]
df_summary["BS PnL"] = df_summary["BS1"] - df_summary["BS0"]


#Marginal PnLs
df_summary["BS ds PnL"] = df_summary["BS0_ds"] - df_summary["BS0"]
df_summary["BS dv PnL"] = df_summary["BS0_dv"] - df_summary["BS0"]
df_summary["BS dt PnL"] = df_summary["BS0_dt"] - df_summary["BS0"]

df_summary["BS dsdv PnL"] = df_summary["BS0_dsdv"] - df_summary["BS0_ds"]
df_summary["BS dsdt PnL"] = df_summary["BS0_dsdt"] - df_summary["BS0_ds"]

df_summary["BS dvds PnL"] = df_summary["BS0_dvds"] - df_summary["BS0_dv"]
df_summary["BS dvdt PnL"] = df_summary["BS0_dvdt"] - df_summary["BS0_dv"]

df_summary["BS dtds PnL"] = df_summary["BS0_dtds"] - df_summary["BS0_dt"]
df_summary["BS dtdv PnL"] = df_summary["BS0_dtdv"] - df_summary["BS0_dt"]


df_summary["BS dsdvdt PnL"] = df_summary["BS1"] - df_summary["BS0_dsdv"]
df_summary["BS dsdtdv PnL"] = df_summary["BS1"] - df_summary["BS0_dsdt"]
df_summary["BS dvdsdt PnL"] = df_summary["BS1"] - df_summary["BS0_dvds"]
df_summary["BS dvdtds PnL"] = df_summary["BS1"] - df_summary["BS0_dvdt"]
df_summary["BS dtdsdv PnL"] = df_summary["BS1"] - df_summary["BS0_dtds"]
df_summary["BS dtdvds PnL"] = df_summary["BS1"] - df_summary["BS0_dtdv"]


df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp( summary_dict["r_f"][0] *  ((date(2020,4,7) - date(2020, 4, 3)).days)/365 ) - \
                                                   x["MV0"]), axis=1 )

df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")


#Short Term Option Hedge Pfl

#Market Value
df_ce_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_CE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")
df_pe_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_PE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")

df_ce_mv0 = df_ce_mv[df_ce_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_ce_mv0 = df_ce_mv0[["Strike Price", "Settle Price"]]
df_ce_mv1 = df_ce_mv[df_ce_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_ce_mv1 = df_ce_mv1[["Strike Price", "Settle Price"]]

df_pe_mv0 = df_pe_mv[df_pe_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_pe_mv0 = df_pe_mv0[["Strike Price", "Settle Price"]]
df_pe_mv1 = df_pe_mv[df_pe_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_pe_mv1 = df_pe_mv1[["Strike Price", "Settle Price"]]


moneyness_list = ["ATM", "ITM", "OTM"]

df_mon_list = []
for moneyness in moneyness_list:
    df_ce_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)
    
    
    l2 = len(df_wts_opt.axes[0])
    df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    df_mon_list.append(df_wts_opt)

df_wts = pd.concat(df_mon_list)
l = len(df_wts.axes[0])
S0 = [summary_dict["S0"][0] for i in range(0,l)]
S1 = [summary_dict["S1"][0] for i in range(0,l)]
dt0= [((end_wk - prv_day).days)/365 for i in range(0,l)]
dt1 = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
r = [0.04 for i in range(0,l)]
r_f = [0.04 for i in range(0,l)]
q = [0 for i in range(0,l)]
df_wts["S0"] = S0
df_wts["S1"] = S1
df_wts["dt0"] = dt0
df_wts["dt1"] = dt1
df_wts["r"] = r
df_wts["r_f"] = r_f
df_wts["q"] = q
df_hdg_pfl = df_wts

df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )


# df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )



df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5* x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

hdge_cash0 = []
hdge_cash1 = []

hdg_MV0 = []
hdge_MV1 = []
hdge_BS0 = []
hdge_BS1 = []

hdge_BS0_ds = []
hdge_BS0_dsdv = []
hdge_BS0_dsdt = []

hdge_BS0_dv = []
hdge_BS0_dvds = []
hdge_BS0_dvdt = []

hdge_BS0_dt = []
hdge_BS0_dtds = []
hdge_BS0_dtdv = []

hdge_delta0 = []
hdge_gamma0 = []
hdge_vega0 = []
hdge_volga0 = []
hdge_vanna0 = []
hdge_theta0 = []
hdge_veta0 = []
hdge_speed0 = []
hdge_charm0 = []

hdgPflrealPnL = []
hdgPflrealOptionsPnL = []
hdgPflPnL = []
hdge_deltaPnL = []
hdge_gammaPnL = []
hdge_vegaPnL = []
hdge_volgaPnL = []
hdge_vannaPnL = []
hdge_thetaPnL = []
hdge_vetaPnL = []
hdge_speedPnL = []
hdge_charmPnL = []


for moneyness in moneyness_list:
    df_hdg_sub_pfl = df_hdg_pfl[df_hdg_pfl["Moneyness"] == moneyness]
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "Moneyness": moneyness_list, "MV0": hdg_MV0, "MV1": hdge_MV1, \
                     "BS0":hdge_BS0, "BS1":hdge_BS1, \
                    "BS0_ds":hdge_BS0_ds, "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
                    "BS0_dv":hdge_BS0_dv, "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
                    "BS0_dt":hdge_BS0_dt, "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
                    "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
                    "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
                    "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
                    "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
                    "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
                    "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
                    "Hdge Pfl Realised PnL":hdgPflrealPnL, "Hdge Pfl Realised Options PnL": hdgPflrealOptionsPnL, "Hdge Pfl PnL":hdgPflPnL}


df_summary_hdg =pd.DataFrame(dict_summary_hdg)

df_summary_hdg["BS Options PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0"]

df_summary_hdg["BS ds PnL"] = df_summary_hdg["BS0_ds"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dv PnL"] = df_summary_hdg["BS0_dv"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dt PnL"] = df_summary_hdg["BS0_dt"] - df_summary_hdg["BS0"]

df_summary_hdg["BS dsdv PnL"] = df_summary_hdg["BS0_dsdv"] - df_summary_hdg["BS0_ds"]
df_summary_hdg["BS dsdt PnL"] = df_summary_hdg["BS0_dsdt"] - df_summary_hdg["BS0_ds"]

df_summary_hdg["BS dvds PnL"] = df_summary_hdg["BS0_dvds"] - df_summary_hdg["BS0_dv"]
df_summary_hdg["BS dvdt PnL"] = df_summary_hdg["BS0_dvdt"] - df_summary_hdg["BS0_dv"]

df_summary_hdg["BS dtds PnL"] = df_summary_hdg["BS0_dtds"] - df_summary_hdg["BS0_dt"]
df_summary_hdg["BS dtdv PnL"] = df_summary_hdg["BS0_dtdv"] - df_summary_hdg["BS0_dt"]


df_summary_hdg["BS dsdvdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdv"]
df_summary_hdg["BS dsdtdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdt"]
df_summary_hdg["BS dvdsdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvds"]
df_summary_hdg["BS dvdtds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvdt"]
df_summary_hdg["BS dtdsdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtds"]
df_summary_hdg["BS dtdvds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtdv"]



df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# # # print(df_summary_hdg)


In [118]:
###########################################################
# Target Option PnL Attribution - 4th May - NIFTY CALL ##
###########################################################

prv_day = date(2020,4,30)
nxt_day = date(2020,5,4)
begin_wk = date(2020,4,30)
end_wk = date(2020,5,7)
end_mth = date(2020,5,28)
prev_price_S0 = 9859.9
next_price_S1 = 9293.5

summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
                "S0": [prev_price_S0 for i in range(0,3)], "S1": [next_price_S1 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
                 "K": [9900, 9000, 11000], "MV0": [280.65, 920.35, 16.75], "MV1": [116.8, 557.65, 9.15], \
                  "dt0": [((end_mth - prv_day).days)/365 for i in range(0,3)], \
                  "dt1": [((end_mth - nxt_day).days)/365 for i in range(0,3)]}

#Incremental Analysis

#Long Term Option
df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_ds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_summary["BS0_dv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_dt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


# Greeks
df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

#PnL
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["iv1"] - x["iv0"]) *(x["S1"] - x["S0"]), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * (x["dt0"] - x["dt1"]) , axis=1 )
df_summary["veta PnL"] = df_summary.apply(lambda x: x["veta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S1"] - x["S0"]),3)), axis=1 )
df_summary["charm PnL"] = df_summary.apply(lambda x: x["charm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )

                                         
df_summary["Target Option Realised PnL"] = df_summary["MV1"] - df_summary["MV0"]
df_summary["BS PnL"] = df_summary["BS1"] - df_summary["BS0"]


#Marginal PnLs
df_summary["BS ds PnL"] = df_summary["BS0_ds"] - df_summary["BS0"]
df_summary["BS dv PnL"] = df_summary["BS0_dv"] - df_summary["BS0"]
df_summary["BS dt PnL"] = df_summary["BS0_dt"] - df_summary["BS0"]

df_summary["BS dsdv PnL"] = df_summary["BS0_dsdv"] - df_summary["BS0_ds"]
df_summary["BS dsdt PnL"] = df_summary["BS0_dsdt"] - df_summary["BS0_ds"]

df_summary["BS dvds PnL"] = df_summary["BS0_dvds"] - df_summary["BS0_dv"]
df_summary["BS dvdt PnL"] = df_summary["BS0_dvdt"] - df_summary["BS0_dv"]

df_summary["BS dtds PnL"] = df_summary["BS0_dtds"] - df_summary["BS0_dt"]
df_summary["BS dtdv PnL"] = df_summary["BS0_dtdv"] - df_summary["BS0_dt"]


df_summary["BS dsdvdt PnL"] = df_summary["BS1"] - df_summary["BS0_dsdv"]
df_summary["BS dsdtdv PnL"] = df_summary["BS1"] - df_summary["BS0_dsdt"]
df_summary["BS dvdsdt PnL"] = df_summary["BS1"] - df_summary["BS0_dvds"]
df_summary["BS dvdtds PnL"] = df_summary["BS1"] - df_summary["BS0_dvdt"]
df_summary["BS dtdsdv PnL"] = df_summary["BS1"] - df_summary["BS0_dtds"]
df_summary["BS dtdvds PnL"] = df_summary["BS1"] - df_summary["BS0_dtdv"]


df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp( summary_dict["r_f"][0] *  ((date(2020,4,7) - date(2020, 4, 3)).days)/365 ) - \
                                                   x["MV0"]), axis=1 )

df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + ".csv")


#Short Term Option Hedge Pfl

#Market Value
df_ce_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_CE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")
df_pe_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_PE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")


df_ce_mv0 = df_ce_mv[df_ce_mv["Date"] == (prv_day).strftime("%d-%b-%y")]
df_ce_mv0 = df_ce_mv0[["Strike Price", "Settle Price"]]
df_ce_mv1 = df_ce_mv[df_ce_mv["Date"] == (nxt_day).strftime("%d-%b-%y")]
df_ce_mv1 = df_ce_mv1[["Strike Price", "Settle Price"]]

df_pe_mv0 = df_pe_mv[df_pe_mv["Date"] == (prv_day).strftime("%d-%b-%y")]
df_pe_mv0 = df_pe_mv0[["Strike Price", "Settle Price"]]
df_pe_mv1 = df_pe_mv[df_pe_mv["Date"] == (nxt_day).strftime("%d-%b-%y")]
df_pe_mv1 = df_pe_mv1[["Strike Price", "Settle Price"]]

# print(df_ce_mv0)

moneyness_list = ["ATM", "ITM", "OTM"]

df_mon_list = []
for moneyness in moneyness_list:
    df_ce_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)
    
    
    l2 = len(df_wts_opt.axes[0])
    df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    df_mon_list.append(df_wts_opt)

df_wts = pd.concat(df_mon_list)
l = len(df_wts.axes[0])
S0 = [summary_dict["S0"][0] for i in range(0,l)]
S1 = [summary_dict["S1"][0] for i in range(0,l)]
dt0= [((end_wk - prv_day).days)/365 for i in range(0,l)]
dt1 = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
r = [0.04 for i in range(0,l)]
r_f = [0.04 for i in range(0,l)]
q = [0 for i in range(0,l)]
df_wts["S0"] = S0
df_wts["S1"] = S1
df_wts["dt0"] = dt0
df_wts["dt1"] = dt1
df_wts["r"] = r
df_wts["r_f"] = r_f
df_wts["q"] = q
df_hdg_pfl = df_wts

df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

hdge_cash0 = []
hdge_cash1 = []

hdg_MV0 = []
hdge_MV1 = []
hdge_BS0 = []
hdge_BS1 = []

hdge_BS0_ds = []
hdge_BS0_dsdv = []
hdge_BS0_dsdt = []

hdge_BS0_dv = []
hdge_BS0_dvds = []
hdge_BS0_dvdt = []

hdge_BS0_dt = []
hdge_BS0_dtds = []
hdge_BS0_dtdv = []

hdge_delta0 = []
hdge_gamma0 = []
hdge_vega0 = []
hdge_volga0 = []
hdge_vanna0 = []
hdge_theta0 = []
hdge_veta0 = []
hdge_speed0 = []
hdge_charm0 = []

hdgPflrealPnL = []
hdgPflrealOptionsPnL = []
hdgPflPnL = []
hdge_deltaPnL = []
hdge_gammaPnL = []
hdge_vegaPnL = []
hdge_volgaPnL = []
hdge_vannaPnL = []
hdge_thetaPnL = []
hdge_vetaPnL = []
hdge_speedPnL = []
hdge_charmPnL = []


for moneyness in moneyness_list:
    df_hdg_sub_pfl = df_hdg_pfl[df_hdg_pfl["Moneyness"] == moneyness]
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "Moneyness": moneyness_list, "MV0": hdg_MV0, "MV1": hdge_MV1, \
                     "BS0":hdge_BS0, "BS1":hdge_BS1, \
                    "BS0_ds":hdge_BS0_ds, "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
                    "BS0_dv":hdge_BS0_dv, "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
                    "BS0_dt":hdge_BS0_dt, "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
                    "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
                    "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
                    "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
                    "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
                    "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
                    "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
                    "Hdge Pfl Realised PnL":hdgPflrealPnL, "Hdge Pfl Realised Options PnL": hdgPflrealOptionsPnL, "Hdge Pfl PnL":hdgPflPnL}


df_summary_hdg =pd.DataFrame(dict_summary_hdg)

df_summary_hdg["BS Options PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0"]

df_summary_hdg["BS ds PnL"] = df_summary_hdg["BS0_ds"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dv PnL"] = df_summary_hdg["BS0_dv"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dt PnL"] = df_summary_hdg["BS0_dt"] - df_summary_hdg["BS0"]

df_summary_hdg["BS dsdv PnL"] = df_summary_hdg["BS0_dsdv"] - df_summary_hdg["BS0_ds"]
df_summary_hdg["BS dsdt PnL"] = df_summary_hdg["BS0_dsdt"] - df_summary_hdg["BS0_ds"]

df_summary_hdg["BS dvds PnL"] = df_summary_hdg["BS0_dvds"] - df_summary_hdg["BS0_dv"]
df_summary_hdg["BS dvdt PnL"] = df_summary_hdg["BS0_dvdt"] - df_summary_hdg["BS0_dv"]

df_summary_hdg["BS dtds PnL"] = df_summary_hdg["BS0_dtds"] - df_summary_hdg["BS0_dt"]
df_summary_hdg["BS dtdv PnL"] = df_summary_hdg["BS0_dtdv"] - df_summary_hdg["BS0_dt"]


df_summary_hdg["BS dsdvdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdv"]
df_summary_hdg["BS dsdtdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdt"]
df_summary_hdg["BS dvdsdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvds"]
df_summary_hdg["BS dvdtds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvdt"]
df_summary_hdg["BS dtdsdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtds"]
df_summary_hdg["BS dtdvds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtdv"]



df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# # # print(df_summary_hdg)


In [5]:
###########################################################
# Target Option PnL Attribution - 30th Mar - NIFTY CALL ##
###########################################################

prv_day = date(2020,3,27)
nxt_day = date(2020,3,30)
begin_wk = date(2020,3,26)
end_wk = date(2020,4,1)
end_mth = date(2020,4,30)
prev_price_S0 = 8660.25
next_price_S1 = 8281.1

summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
                "S0": [prev_price_S0 for i in range(0,3)], "S1": [next_price_S1 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
                 "K": [8600, 8000, 9500], "MV0": [713.95, 1125.3, 275.45], "MV1": [513.95, 847.45, 195.85], \
                  "dt0": [((end_mth - prv_day).days)/365 for i in range(0,3)], \
                  "dt1": [((end_mth - nxt_day).days)/365 for i in range(0,3)]}

#Incremental Analysis

#Long Term Option
df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_ds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_summary["BS0_dv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_dt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


# Greeks
df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

# df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )


#PnL
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["iv1"] - x["iv0"]) *(x["S1"] - x["S0"]), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * (x["dt0"] - x["dt1"]) , axis=1 )
df_summary["veta PnL"] = df_summary.apply(lambda x: x["veta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S1"] - x["S0"]),3)), axis=1 )
df_summary["charm PnL"] = df_summary.apply(lambda x: x["charm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )

                                         
df_summary["Target Option Realised PnL"] = df_summary["MV1"] - df_summary["MV0"]
df_summary["BS PnL"] = df_summary["BS1"] - df_summary["BS0"]


#Marginal PnLs
df_summary["BS ds PnL"] = df_summary["BS0_ds"] - df_summary["BS0"]
df_summary["BS dv PnL"] = df_summary["BS0_dv"] - df_summary["BS0"]
df_summary["BS dt PnL"] = df_summary["BS0_dt"] - df_summary["BS0"]

df_summary["BS dsdv PnL"] = df_summary["BS0_dsdv"] - df_summary["BS0_ds"]
df_summary["BS dsdt PnL"] = df_summary["BS0_dsdt"] - df_summary["BS0_ds"]

df_summary["BS dvds PnL"] = df_summary["BS0_dvds"] - df_summary["BS0_dv"]
df_summary["BS dvdt PnL"] = df_summary["BS0_dvdt"] - df_summary["BS0_dv"]

df_summary["BS dtds PnL"] = df_summary["BS0_dtds"] - df_summary["BS0_dt"]
df_summary["BS dtdv PnL"] = df_summary["BS0_dtdv"] - df_summary["BS0_dt"]


df_summary["BS dsdvdt PnL"] = df_summary["BS1"] - df_summary["BS0_dsdv"]
df_summary["BS dsdtdv PnL"] = df_summary["BS1"] - df_summary["BS0_dsdt"]
df_summary["BS dvdsdt PnL"] = df_summary["BS1"] - df_summary["BS0_dvds"]
df_summary["BS dvdtds PnL"] = df_summary["BS1"] - df_summary["BS0_dvdt"]
df_summary["BS dtdsdv PnL"] = df_summary["BS1"] - df_summary["BS0_dtds"]
df_summary["BS dtdvds PnL"] = df_summary["BS1"] - df_summary["BS0_dtdv"]


df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp( summary_dict["r_f"][0] *  ((date(2020,4,7) - date(2020, 4, 3)).days)/365 ) - \
                                                   x["MV0"]), axis=1 )

df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")


#Short Term Option Hedge Pfl







#Market Value
df_ce_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_CE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")
df_pe_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_PE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")

df_ce_mv0 = df_ce_mv[df_ce_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_ce_mv0 = df_ce_mv0[["Strike Price", "Settle Price"]]
df_ce_mv1 = df_ce_mv[df_ce_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_ce_mv1 = df_ce_mv1[["Strike Price", "Settle Price"]]

df_pe_mv0 = df_pe_mv[df_pe_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_pe_mv0 = df_pe_mv0[["Strike Price", "Settle Price"]]
df_pe_mv1 = df_pe_mv[df_pe_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_pe_mv1 = df_pe_mv1[["Strike Price", "Settle Price"]]


moneyness_list = ["ATM", "ITM", "OTM"]

df_mon_list = []
for moneyness in moneyness_list:
    df_ce_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)
    
    
    l2 = len(df_wts_opt.axes[0])
    df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    df_mon_list.append(df_wts_opt)

df_wts = pd.concat(df_mon_list)
l = len(df_wts.axes[0])
S0 = [summary_dict["S0"][0] for i in range(0,l)]
S1 = [summary_dict["S1"][0] for i in range(0,l)]
dt0= [((end_wk - prv_day).days)/365 for i in range(0,l)]
dt1 = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
r = [0.04 for i in range(0,l)]
r_f = [0.04 for i in range(0,l)]
q = [0 for i in range(0,l)]
df_wts["S0"] = S0
df_wts["S1"] = S1
df_wts["dt0"] = dt0
df_wts["dt1"] = dt1
df_wts["r"] = r
df_wts["r_f"] = r_f
df_wts["q"] = q
df_hdg_pfl = df_wts

df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )


# df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )



df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5* x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

hdge_cash0 = []
hdge_cash1 = []

hdg_MV0 = []
hdge_MV1 = []
hdge_BS0 = []
hdge_BS1 = []

hdge_BS0_ds = []
hdge_BS0_dsdv = []
hdge_BS0_dsdt = []

hdge_BS0_dv = []
hdge_BS0_dvds = []
hdge_BS0_dvdt = []

hdge_BS0_dt = []
hdge_BS0_dtds = []
hdge_BS0_dtdv = []

hdge_delta0 = []
hdge_gamma0 = []
hdge_vega0 = []
hdge_volga0 = []
hdge_vanna0 = []
hdge_theta0 = []
hdge_veta0 = []
hdge_speed0 = []
hdge_charm0 = []

hdgPflrealPnL = []
hdgPflrealOptionsPnL = []
hdgPflPnL = []
hdge_deltaPnL = []
hdge_gammaPnL = []
hdge_vegaPnL = []
hdge_volgaPnL = []
hdge_vannaPnL = []
hdge_thetaPnL = []
hdge_vetaPnL = []
hdge_speedPnL = []
hdge_charmPnL = []


for moneyness in moneyness_list:
    df_hdg_sub_pfl = df_hdg_pfl[df_hdg_pfl["Moneyness"] == moneyness]
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "Moneyness": moneyness_list, "MV0": hdg_MV0, "MV1": hdge_MV1, \
                     "BS0":hdge_BS0, "BS1":hdge_BS1, \
                    "BS0_ds":hdge_BS0_ds, "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
                    "BS0_dv":hdge_BS0_dv, "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
                    "BS0_dt":hdge_BS0_dt, "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
                    "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
                    "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
                    "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
                    "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
                    "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
                    "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
                    "Hdge Pfl Realised PnL":hdgPflrealPnL, "Hdge Pfl Realised Options PnL": hdgPflrealOptionsPnL, "Hdge Pfl PnL":hdgPflPnL}


df_summary_hdg =pd.DataFrame(dict_summary_hdg)

df_summary_hdg["BS Options PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0"]

df_summary_hdg["BS ds PnL"] = df_summary_hdg["BS0_ds"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dv PnL"] = df_summary_hdg["BS0_dv"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dt PnL"] = df_summary_hdg["BS0_dt"] - df_summary_hdg["BS0"]

df_summary_hdg["BS dsdv PnL"] = df_summary_hdg["BS0_dsdv"] - df_summary_hdg["BS0_ds"]
df_summary_hdg["BS dsdt PnL"] = df_summary_hdg["BS0_dsdt"] - df_summary_hdg["BS0_ds"]

df_summary_hdg["BS dvds PnL"] = df_summary_hdg["BS0_dvds"] - df_summary_hdg["BS0_dv"]
df_summary_hdg["BS dvdt PnL"] = df_summary_hdg["BS0_dvdt"] - df_summary_hdg["BS0_dv"]

df_summary_hdg["BS dtds PnL"] = df_summary_hdg["BS0_dtds"] - df_summary_hdg["BS0_dt"]
df_summary_hdg["BS dtdv PnL"] = df_summary_hdg["BS0_dtdv"] - df_summary_hdg["BS0_dt"]


df_summary_hdg["BS dsdvdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdv"]
df_summary_hdg["BS dsdtdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdt"]
df_summary_hdg["BS dvdsdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvds"]
df_summary_hdg["BS dvdtds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvdt"]
df_summary_hdg["BS dtdsdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtds"]
df_summary_hdg["BS dtdvds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtdv"]



df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# # # print(df_summary_hdg)


In [19]:
###########################################################
# Target Option PnL Attribution - 12th Mar - NIFTY CALL ##
###########################################################

def brentqc(error_func, min_iv, max_iv, MV0, S0, K, r, r_f, dt0, option_type):
    try:
        iv = brentq(error_func, min_iv, max_iv, args=(MV0, S0, K, r, r_f, dt0, option_type), \
               xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    except:
         iv = 0.000001
    return iv

def BSM_put_vec_with_div(s_t, k, r, r_f, vol, dt):
    if (dt == 0):
        return np.maximum(k-s_t, 0)
    elif (vol == 0):
        return np.maximum(k - s_t * np.exp(r*dt), 0)
    else:
        d_1_c = (1 / (vol * (dt ** 0.5))) 
        d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
        d_1 = d_1_c * d1_log
        d_2 = d_1 - vol * (dt ** 0.5)
        q = r_f - r      
        return ((k * np.exp(-r_f*dt) * norm.cdf(-d_2)) - np.multiply(s_t, norm.cdf(-d_1)) * np.exp(-q * dt) )


def BSM_call_vec_with_div(s_t, k, r, r_f, vol, dt):
    if (dt == 0):
        return np.maximum(s_t-k, 0)
    elif (vol == 0):
        return np.maximum(s_t * np.exp(r*dt) - k, 0)
    else:
        d_1_c = (1 / (vol * (dt ** 0.5))) 
        d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
        d_1 = d_1_c * d1_log
        d_2 = d_1 - vol * (dt ** 0.5)
        q = r_f - r
        return (np.multiply(norm.cdf(d_1),  s_t) * np.exp(-q * dt) - (norm.cdf(d_2) * k * np.exp(-r_f * dt)))



prv_day = date(2020,3,11)
nxt_day = date(2020,3,12)
begin_wk = date(2020,3,5)
end_wk = date(2020,3,12)
end_mth = date(2020,3,26)
prev_price_S0 = 10458.4
next_price_S1 = 9590.15
risk_free_rate = 0.04
summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
                "S0": [prev_price_S0 for i in range(0,3)], "S1": [next_price_S1 for i in range(0,3)], "r_f": [risk_free_rate for i in range(0,3)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [risk_free_rate for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
                 "K": [11650, 10500, 12900], "MV0": [6.7, 249.55, 3.45], "MV1": [9.65, 64.7, 3.25], \
                  "dt0": [((end_mth - prv_day).days)/365 for i in range(0,3)], \
                  "dt1": [((end_mth - nxt_day).days)/365 for i in range(0,3)]}

#Incremental Analysis

#Long Term Option
df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentqc(error_func, 0.00001, 50, x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentqc(error_func, 0.00001, 50, x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_ds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_summary["BS0_dv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_dt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


# Greeks
df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

# df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )


#PnL
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["iv1"] - x["iv0"]) *(x["S1"] - x["S0"]), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * (x["dt0"] - x["dt1"]) , axis=1 )
df_summary["veta PnL"] = df_summary.apply(lambda x: x["veta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S1"] - x["S0"]),3)), axis=1 )
df_summary["charm PnL"] = df_summary.apply(lambda x: x["charm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )

                                         
df_summary["Target Option Realised PnL"] = df_summary["MV1"] - df_summary["MV0"]
df_summary["BS PnL"] = df_summary["BS1"] - df_summary["BS0"]


#Marginal PnLs
df_summary["BS ds PnL"] = df_summary["BS0_ds"] - df_summary["BS0"]
df_summary["BS dv PnL"] = df_summary["BS0_dv"] - df_summary["BS0"]
df_summary["BS dt PnL"] = df_summary["BS0_dt"] - df_summary["BS0"]

df_summary["BS dsdv PnL"] = df_summary["BS0_dsdv"] - df_summary["BS0_ds"]
df_summary["BS dsdt PnL"] = df_summary["BS0_dsdt"] - df_summary["BS0_ds"]

df_summary["BS dvds PnL"] = df_summary["BS0_dvds"] - df_summary["BS0_dv"]
df_summary["BS dvdt PnL"] = df_summary["BS0_dvdt"] - df_summary["BS0_dv"]

df_summary["BS dtds PnL"] = df_summary["BS0_dtds"] - df_summary["BS0_dt"]
df_summary["BS dtdv PnL"] = df_summary["BS0_dtdv"] - df_summary["BS0_dt"]


df_summary["BS dsdvdt PnL"] = df_summary["BS1"] - df_summary["BS0_dsdv"]
df_summary["BS dsdtdv PnL"] = df_summary["BS1"] - df_summary["BS0_dsdt"]
df_summary["BS dvdsdt PnL"] = df_summary["BS1"] - df_summary["BS0_dvds"]
df_summary["BS dvdtds PnL"] = df_summary["BS1"] - df_summary["BS0_dvdt"]
df_summary["BS dtdsdv PnL"] = df_summary["BS1"] - df_summary["BS0_dtds"]
df_summary["BS dtdvds PnL"] = df_summary["BS1"] - df_summary["BS0_dtdv"]


df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp( summary_dict["r_f"][0] *  ((date(2020,4,7) - date(2020, 4, 3)).days)/365 ) - \
                                                   x["MV0"]), axis=1 )

# df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")


#Short Term Option Hedge Pfl







#Market Value
df_ce_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_CE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")
df_pe_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_PE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")

df_ce_mv0 = df_ce_mv[df_ce_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_ce_mv0 = df_ce_mv0[["Strike Price", "Settle Price"]]
df_ce_mv1 = df_ce_mv[df_ce_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_ce_mv1 = df_ce_mv1[["Strike Price", "Settle Price"]]

df_pe_mv0 = df_pe_mv[df_pe_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_pe_mv0 = df_pe_mv0[["Strike Price", "Settle Price"]]
df_pe_mv1 = df_pe_mv[df_pe_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_pe_mv1 = df_pe_mv1[["Strike Price", "Settle Price"]]


moneyness_list = ["ATM", "ITM", "OTM"]

df_mon_list = []
for moneyness in moneyness_list:
    df_ce_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)
    
    
    l2 = len(df_wts_opt.axes[0])
    df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    df_mon_list.append(df_wts_opt)

df_wts = pd.concat(df_mon_list)
l = len(df_wts.axes[0])
S0 = [summary_dict["S0"][0] for i in range(0,l)]
S1 = [summary_dict["S1"][0] for i in range(0,l)]
dt0= [((end_wk - prv_day).days)/365 for i in range(0,l)]
dt1 = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
r = [risk_free_rate for i in range(0,l)]
r_f = [risk_free_rate for i in range(0,l)]
q = [0 for i in range(0,l)]
df_wts["S0"] = S0
df_wts["S1"] = S1
df_wts["dt0"] = dt0
df_wts["dt1"] = dt1
df_wts["r"] = r
df_wts["r_f"] = r_f
df_wts["q"] = q
df_hdg_pfl = df_wts
# print(df_hdg_pfl)

df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: brentqc(error_func, 0.00001, 50, x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), axis=1 )

df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: brentqc(error_func, 0.00001, 50, x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]) if x["dt1"] > 0 else 0, axis=1 )

df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: np.maximum(x["S1"]- x["K"], 0) if x["Option Type"] == "CE" \
                                            else np.maximum(x["K"] - x["S1"], 0), axis=1 )


df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )





df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5* x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

hdge_cash0 = []
hdge_cash1 = []

hdg_MV0 = []
hdge_MV1 = []
hdge_BS0 = []
hdge_BS1 = []

hdge_BS0_ds = []
hdge_BS0_dsdv = []
hdge_BS0_dsdt = []

hdge_BS0_dv = []
hdge_BS0_dvds = []
hdge_BS0_dvdt = []

hdge_BS0_dt = []
hdge_BS0_dtds = []
hdge_BS0_dtdv = []

hdge_delta0 = []
hdge_gamma0 = []
hdge_vega0 = []
hdge_volga0 = []
hdge_vanna0 = []
hdge_theta0 = []
hdge_veta0 = []
hdge_speed0 = []
hdge_charm0 = []

hdgPflrealPnL = []
hdgPflrealOptionsPnL = []
hdgPflPnL = []
hdge_deltaPnL = []
hdge_gammaPnL = []
hdge_vegaPnL = []
hdge_volgaPnL = []
hdge_vannaPnL = []
hdge_thetaPnL = []
hdge_vetaPnL = []
hdge_speedPnL = []
hdge_charmPnL = []


for moneyness in moneyness_list:
    df_hdg_sub_pfl = df_hdg_pfl[df_hdg_pfl["Moneyness"] == moneyness]
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "Moneyness": moneyness_list, "MV0": hdg_MV0, "MV1": hdge_MV1, \
                     "BS0":hdge_BS0, "BS1":hdge_BS1, \
                    "BS0_ds":hdge_BS0_ds, \
                    "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
                    "BS0_dv":hdge_BS0_dv, \
                    "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
                    "BS0_dt":hdge_BS0_dt, \
                    "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
                    "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
                    "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
                    "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
                    "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
                    "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
                    "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
                    "Hdge Pfl Realised PnL":hdgPflrealPnL, "Hdge Pfl Realised Options PnL": hdgPflrealOptionsPnL, "Hdge Pfl PnL":hdgPflPnL}


df_summary_hdg =pd.DataFrame(dict_summary_hdg)

df_summary_hdg["BS Options PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0"]

df_summary_hdg["BS ds PnL"] = df_summary_hdg["BS0_ds"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dv PnL"] = df_summary_hdg["BS0_dv"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dt PnL"] = df_summary_hdg["BS0_dt"] - df_summary_hdg["BS0"]

df_summary_hdg["BS dsdv PnL"] = df_summary_hdg["BS0_dsdv"] - df_summary_hdg["BS0_ds"]
df_summary_hdg["BS dsdt PnL"] = df_summary_hdg["BS0_dsdt"] - df_summary_hdg["BS0_ds"]

df_summary_hdg["BS dvds PnL"] = df_summary_hdg["BS0_dvds"] - df_summary_hdg["BS0_dv"]
df_summary_hdg["BS dvdt PnL"] = df_summary_hdg["BS0_dvdt"] - df_summary_hdg["BS0_dv"]

df_summary_hdg["BS dtds PnL"] = df_summary_hdg["BS0_dtds"] - df_summary_hdg["BS0_dt"]
df_summary_hdg["BS dtdv PnL"] = df_summary_hdg["BS0_dtdv"] - df_summary_hdg["BS0_dt"]


df_summary_hdg["BS dsdvdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdv"]
df_summary_hdg["BS dsdtdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdt"]
df_summary_hdg["BS dvdsdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvds"]
df_summary_hdg["BS dvdtds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvdt"]
df_summary_hdg["BS dtdsdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtds"]
df_summary_hdg["BS dtdvds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtdv"]



df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# # # df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# # # df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# # # # # print(df_summary_hdg)


In [27]:
# #ATM case


                    
df_hedge_ATM = df_hdg_pfl[df_hdg_pfl["Moneyness"] == "ATM"]
# for i in range(0, df_hedge_ATM.shape[0]):
# #     if (i == 18):
# #         mv0 = df_hedge_ATM.loc[i,"MV0"]
# #         K = df_hedge_ATM.loc[i,"K"]
# #         option_type = df_hedge_ATM.loc[i,"Option Type"]
# #         print(BSM_put_vec_with_div(10458.4, 10700, 0.04, 0.04, 0.00001, 0.00274))
# #         print(BSM_put_vec_with_div(10458.4, 10700, 0.04, 0.04, 50, 0.00274))
# #         print(mv0)
# #         iv = brentq(error_func, 0.00001, 50, args=(mv0, 10458.4, K, 0.04, 0.04, 0.00274, option_type), \
# #                    xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
# #         print(i, iv)
#     mv0 = df_hedge_ATM.loc[i,"MV0"]
#     K = df_hedge_ATM.loc[i,"K"]
#     option_type = df_hedge_ATM.loc[i,"Option Type"]

#     iv = brentqc(error_func, 0.00001, 50, mv0, 10458.4, K, 0.04, 0.04, 0.00274, option_type)

#     print(i, iv)
# 

print(df_hedge_ATM.loc[19,:])
print(df_hedge_ATM.loc[19, ["MV0", "MV1", "S0", "S1", "K", "Option Type", "Moneyness", "Pfl_weights"]])
# print(df_hedge_ATM.loc[19, []])
print(BSM_put_vec_with_div(10458.4, 10800, 0.04, 0.04, 0.0000000000000001, 0.00274))
print(BSM_put_vec_with_div(10458.4, 10800, 0.04, 0.04, 50, 0.00274))
print(np.maximum(10800-9590.15,0))

Option Type          PE
K                 10800
Pfl_weights    0.019446
MV0               337.4
MV1             9590.15
                 ...   
wvanna PnL          0.0
wtheta PnL     0.026151
wveta PnL          -0.0
wspeed PnL         -0.0
wcharm PnL          0.0
Name: 19, Length: 71, dtype: object
MV0               337.4
MV1             9590.15
S0              10458.4
S1              9590.15
K                 10800
Option Type          PE
Moneyness           ATM
Pfl_weights    0.019446
Name: 19, dtype: object
340.4163848632943
8772.704787593586
1209.8500000000004


In [16]:
prv_day = date(2020,2,27)
nxt_day = date(2020,2,28)
begin_wk = date(2020,2,27)
end_wk = date(2020,3,5)
end_mth = date(2020,3,26)
prev_price_S0 = 11633.3
next_price_S1 = 11201.75

print(BSM_call_vec_with_div(11201.75, 11650, 0.04, 0.04, 50, ((end_mth - nxt_day).days)/365))
print(((end_mth - nxt_day).days)/365)

11201.749999880229
0.07397260273972603


In [13]:
###########################################################
# Target Option PnL Attribution - 30th Mar - NIFTY CALL ##
###########################################################

prv_day = date(2020,2,27)
nxt_day = date(2020,2,28)
begin_wk = date(2020,2,27)
end_wk = date(2020,3,5)
end_mth = date(2020,3,26)
prev_price_S0 = 11633.3
next_price_S1 = 11201.75

summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
                "S0": [prev_price_S0 for i in range(0,3)], "S1": [next_price_S1 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
                 "K": [11650, 10500, 12900], "MV0": [75.2, 731.95, 4.05], "MV1": [203.55, 1135.5, 4.85], \
                  "dt0": [((end_mth - prv_day).days)/365 for i in range(0,3)], \
                  "dt1": [((end_mth - nxt_day).days)/365 for i in range(0,3)]}

#Incremental Analysis

#Long Term Option
df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_ds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_summary["BS0_dv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_dt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


# Greeks
df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

# df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )


#PnL
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["iv1"] - x["iv0"]) *(x["S1"] - x["S0"]), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * (x["dt0"] - x["dt1"]) , axis=1 )
df_summary["veta PnL"] = df_summary.apply(lambda x: x["veta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S1"] - x["S0"]),3)), axis=1 )
df_summary["charm PnL"] = df_summary.apply(lambda x: x["charm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )

                                         
df_summary["Target Option Realised PnL"] = df_summary["MV1"] - df_summary["MV0"]
df_summary["BS PnL"] = df_summary["BS1"] - df_summary["BS0"]


#Marginal PnLs
df_summary["BS ds PnL"] = df_summary["BS0_ds"] - df_summary["BS0"]
df_summary["BS dv PnL"] = df_summary["BS0_dv"] - df_summary["BS0"]
df_summary["BS dt PnL"] = df_summary["BS0_dt"] - df_summary["BS0"]

df_summary["BS dsdv PnL"] = df_summary["BS0_dsdv"] - df_summary["BS0_ds"]
df_summary["BS dsdt PnL"] = df_summary["BS0_dsdt"] - df_summary["BS0_ds"]

df_summary["BS dvds PnL"] = df_summary["BS0_dvds"] - df_summary["BS0_dv"]
df_summary["BS dvdt PnL"] = df_summary["BS0_dvdt"] - df_summary["BS0_dv"]

df_summary["BS dtds PnL"] = df_summary["BS0_dtds"] - df_summary["BS0_dt"]
df_summary["BS dtdv PnL"] = df_summary["BS0_dtdv"] - df_summary["BS0_dt"]


df_summary["BS dsdvdt PnL"] = df_summary["BS1"] - df_summary["BS0_dsdv"]
df_summary["BS dsdtdv PnL"] = df_summary["BS1"] - df_summary["BS0_dsdt"]
df_summary["BS dvdsdt PnL"] = df_summary["BS1"] - df_summary["BS0_dvds"]
df_summary["BS dvdtds PnL"] = df_summary["BS1"] - df_summary["BS0_dvdt"]
df_summary["BS dtdsdv PnL"] = df_summary["BS1"] - df_summary["BS0_dtds"]
df_summary["BS dtdvds PnL"] = df_summary["BS1"] - df_summary["BS0_dtdv"]


df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp( summary_dict["r_f"][0] *  ((date(2020,4,7) - date(2020, 4, 3)).days)/365 ) - \
                                                   x["MV0"]), axis=1 )

df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")


#Short Term Option Hedge Pfl







#Market Value
df_ce_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_CE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")
df_pe_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_PE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")

df_ce_mv0 = df_ce_mv[df_ce_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_ce_mv0 = df_ce_mv0[["Strike Price", "Settle Price"]]
df_ce_mv1 = df_ce_mv[df_ce_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_ce_mv1 = df_ce_mv1[["Strike Price", "Settle Price"]]

df_pe_mv0 = df_pe_mv[df_pe_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
df_pe_mv0 = df_pe_mv0[["Strike Price", "Settle Price"]]
df_pe_mv1 = df_pe_mv[df_pe_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
df_pe_mv1 = df_pe_mv1[["Strike Price", "Settle Price"]]


# moneyness_list = ["ATM", "ITM", "OTM"]
moneyness_list = ["ATM"]
df_mon_list = []
for moneyness in moneyness_list:
    df_ce_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_NIFTY_" + moneyness + "_CE_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)
    
    
    l2 = len(df_wts_opt.axes[0])
    df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    df_mon_list.append(df_wts_opt)

df_wts = pd.concat(df_mon_list)
l = len(df_wts.axes[0])
S0 = [summary_dict["S0"][0] for i in range(0,l)]
S1 = [summary_dict["S1"][0] for i in range(0,l)]
dt0= [((end_wk - prv_day).days)/365 for i in range(0,l)]
dt1 = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
r = [0.04 for i in range(0,l)]
r_f = [0.04 for i in range(0,l)]
q = [0 for i in range(0,l)]
df_wts["S0"] = S0
df_wts["S1"] = S1
df_wts["dt0"] = dt0
df_wts["dt1"] = dt1
df_wts["r"] = r
df_wts["r_f"] = r_f
df_wts["q"] = q
df_hdg_pfl = df_wts

df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )


# df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) if x["Option Type"] == "CE" \
#                                             else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )



df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5* x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

hdge_cash0 = []
hdge_cash1 = []

hdg_MV0 = []
hdge_MV1 = []
hdge_BS0 = []
hdge_BS1 = []

hdge_BS0_ds = []
hdge_BS0_dsdv = []
hdge_BS0_dsdt = []

hdge_BS0_dv = []
hdge_BS0_dvds = []
hdge_BS0_dvdt = []

hdge_BS0_dt = []
hdge_BS0_dtds = []
hdge_BS0_dtdv = []

hdge_delta0 = []
hdge_gamma0 = []
hdge_vega0 = []
hdge_volga0 = []
hdge_vanna0 = []
hdge_theta0 = []
hdge_veta0 = []
hdge_speed0 = []
hdge_charm0 = []

hdgPflrealPnL = []
hdgPflrealOptionsPnL = []
hdgPflPnL = []
hdge_deltaPnL = []
hdge_gammaPnL = []
hdge_vegaPnL = []
hdge_volgaPnL = []
hdge_vannaPnL = []
hdge_thetaPnL = []
hdge_vetaPnL = []
hdge_speedPnL = []
hdge_charmPnL = []


for moneyness in moneyness_list:
    df_hdg_sub_pfl = df_hdg_pfl[df_hdg_pfl["Moneyness"] == moneyness]
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "Moneyness": moneyness_list, "MV0": hdg_MV0, "MV1": hdge_MV1, \
                     "BS0":hdge_BS0, "BS1":hdge_BS1, \
                    "BS0_ds":hdge_BS0_ds, "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
                    "BS0_dv":hdge_BS0_dv, "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
                    "BS0_dt":hdge_BS0_dt, "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
                    "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
                    "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
                    "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
                    "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
                    "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
                    "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
                    "Hdge Pfl Realised PnL":hdgPflrealPnL, "Hdge Pfl Realised Options PnL": hdgPflrealOptionsPnL, "Hdge Pfl PnL":hdgPflPnL}


df_summary_hdg =pd.DataFrame(dict_summary_hdg)

df_summary_hdg["BS Options PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0"]

df_summary_hdg["BS ds PnL"] = df_summary_hdg["BS0_ds"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dv PnL"] = df_summary_hdg["BS0_dv"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dt PnL"] = df_summary_hdg["BS0_dt"] - df_summary_hdg["BS0"]

df_summary_hdg["BS dsdv PnL"] = df_summary_hdg["BS0_dsdv"] - df_summary_hdg["BS0_ds"]
df_summary_hdg["BS dsdt PnL"] = df_summary_hdg["BS0_dsdt"] - df_summary_hdg["BS0_ds"]

df_summary_hdg["BS dvds PnL"] = df_summary_hdg["BS0_dvds"] - df_summary_hdg["BS0_dv"]
df_summary_hdg["BS dvdt PnL"] = df_summary_hdg["BS0_dvdt"] - df_summary_hdg["BS0_dv"]

df_summary_hdg["BS dtds PnL"] = df_summary_hdg["BS0_dtds"] - df_summary_hdg["BS0_dt"]
df_summary_hdg["BS dtdv PnL"] = df_summary_hdg["BS0_dtdv"] - df_summary_hdg["BS0_dt"]


df_summary_hdg["BS dsdvdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdv"]
df_summary_hdg["BS dsdtdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdt"]
df_summary_hdg["BS dvdsdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvds"]
df_summary_hdg["BS dvdtds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvdt"]
df_summary_hdg["BS dtdsdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtds"]
df_summary_hdg["BS dtdvds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtdv"]



df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_hdg_pfl.to_csv(out_PnL_path + "pnl_attribution_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# df_summary_hdg.to_csv(out_PnL_path + "pnl_attribution_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")
# # # print(df_summary_hdg)


ValueError: f(a) and f(b) must have different signs

In [15]:
########################################################################
######### Target Option Historical PnL Attribution ####################
#######################################################################

def brentqc(error_func, min_iv, max_iv, MV0, S0, K, r, r_f, dt0, option_type):
    try:
        iv = brentq(error_func, min_iv, max_iv, args=(MV0, S0, K, r, r_f, dt0, option_type), \
               xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    except:
         iv = 0.000001
    return iv

def BSM_put_vec_with_div(s_t, k, r, r_f, vol, dt):
    if (dt == 0):
        return np.maximum(k-s_t, 0)
    elif (vol == 0):
        return np.maximum(k - s_t * np.exp(r*dt), 0)
    else:
        d_1_c = (1 / (vol * (dt ** 0.5))) 
        d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
        d_1 = d_1_c * d1_log
        d_2 = d_1 - vol * (dt ** 0.5)
        q = r_f - r      
        return ((k * np.exp(-r_f*dt) * norm.cdf(-d_2)) - np.multiply(s_t, norm.cdf(-d_1)) * np.exp(-q * dt) )


def BSM_call_vec_with_div(s_t, k, r, r_f, vol, dt):
    if (dt == 0):
        return np.maximum(s_t-k, 0)
    elif (vol == 0):
        return np.maximum(s_t * np.exp(r*dt) - k, 0)
    else:
        d_1_c = (1 / (vol * (dt ** 0.5))) 
        d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
        d_1 = d_1_c * d1_log
        d_2 = d_1 - vol * (dt ** 0.5)
        q = r_f - r
        return (np.multiply(norm.cdf(d_1),  s_t) * np.exp(-q * dt) - (norm.cdf(d_2) * k * np.exp(-r_f * dt)))



def option_price(S, K, r, r_f, iv, dt, option_type):
    if(option_type == "CE"):
        price = BSM_put_vec_with_div(S, K, r, r_f, iv, dt)
    else:
        price = BSM_put_vec_with_div(S, K, r, r_f, iv, dt) 
    return price




# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"
#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"
#Product type to hedge: either "CE" or "PE"
prod_type = "CE"

df_pv_target = pd.read_csv(input_data_path + "F6_" + stock_ident + "_" + prod_moneyness + "_" + prod_type \
                           + "_PnL_attribution.csv")


df_pv_target['Date'] = df_pv_target.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1)
df_pv_target['Date_lag'] = df_pv_target.apply(lambda x: x['Date_lag'].split("-")[0] + "-" + \
                                  x['Date_lag'].split("-")[1]  + "-"  + "20" + x['Date_lag'].split("-")[2] \
                                  if (len(x['Date_lag'].split("-")[2]) == 2) else x['Date_lag'], axis=1)
df_pv_target['Month_Expiry'] = df_pv_target.apply(lambda x: x['Month_Expiry'].split("-")[0] + "-" + \
                                  x['Month_Expiry'].split("-")[1]  + "-"  + "20" + x['Month_Expiry'].split("-")[2] \
                                  if (len(x['Month_Expiry'].split("-")[2]) == 2) else x['Month_Expiry'], axis=1)

df_pv_target['Week_Start'] = df_pv_target.apply(lambda x: x['Week_Start'].split("-")[0] + "-" + \
                                  x['Week_Start'].split("-")[1]  + "-"  + "20" + x['Week_Start'].split("-")[2] \
                                  if (len(x['Week_Start'].split("-")[2]) == 2) else x['Week_Start'], axis=1)
df_pv_target['Week_Expiry'] = df_pv_target.apply(lambda x: x['Week_Expiry'].split("-")[0] + "-" + \
                                  x['Week_Expiry'].split("-")[1]  + "-"  + "20" + x['Week_Expiry'].split("-")[2] \
                                  if (len(x['Week_Expiry'].split("-")[2]) == 2) else x['Week_Expiry'], axis=1)
# 26-Mar-2020
# df_pv_target = df_pv_target[df_pv_target['Week_Start'] == date(2020,4,1).strftime("%d-%b-%Y")] 
# df_pv_target = df_pv_target[df_pv_target['Date_lag'] == date(2020,3,26).strftime("%d-%b-%Y")] 


df_pv_target["r_f"] = df_pv_target["r"]
df_pv_target["dt"] = df_pv_target.apply(lambda x: (datetime.strptime(x['Month_Expiry'], "%d-%b-%Y").date() - datetime.strptime(x['Date'], "%d-%b-%Y").date()).days / 365.0, axis=1 )
df_pv_target["dt_lag"] = df_pv_target.apply(lambda x: (datetime.strptime(x['Month_Expiry'], "%d-%b-%Y").date() - \
                                       datetime.strptime(x['Date_lag'], "%d-%b-%Y").date()).days / 365.0, axis=1 )



df_pv_target["iv"] = df_pv_target.apply(lambda x: brentqc(error_func, 0.0000001, 100, x["Close"], x["S"], x["K"], x["r"], x["r_f"], x["dt"], prod_type), axis=1)

df_pv_target["iv_lag"] = df_pv_target.apply(lambda x: brentqc(error_func, 0.0000001, 50, x["Close_lag"], x["S_lag"], x["K"], x["r"], x["r_f"], x["dt_lag"], prod_type), axis=1)

df_pv_target["delta0"] = df_pv_target.apply(lambda x: call_delta_with_spot_and_div(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["gamma0"] = df_pv_target.apply(lambda x: find_gamma(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["vega0"] = df_pv_target.apply(lambda x: find_vega(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["volga0"] = df_pv_target.apply(lambda x: find_volga(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["vanna0"] = df_pv_target.apply(lambda x: find_vanna(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["theta0"] = df_pv_target.apply(lambda x: find_theta_Call(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["veta0"] = df_pv_target.apply(lambda x: find_veta(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["speed0"] = df_pv_target.apply(lambda x: find_speed(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["charm0"] = df_pv_target.apply(lambda x: find_charm_call(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )

df_pv_target["ds"] = df_pv_target["S"] - df_pv_target["S_lag"]
df_pv_target["t_passed"] = df_pv_target["dt_lag"] - df_pv_target["dt"]
df_pv_target["dvol"] = df_pv_target["iv"] - df_pv_target["iv_lag"]
df_pv_target["Target Option PnL"] = df_pv_target["Close"] - df_pv_target["Close_lag"]

#PnL
df_pv_target["delta PnL"] = df_pv_target.apply(lambda x: x["delta0"] * (x["S"] - x["S_lag"]), axis=1 )
df_pv_target["gamma PnL"] = df_pv_target.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S"] - x["S_lag"])), axis=1 )
df_pv_target["vega PnL"] = df_pv_target.apply(lambda x: x["vega0"] * (x["iv"] - x["iv_lag"]), axis=1 )
df_pv_target["volga PnL"] = df_pv_target.apply(lambda x: 0.5 * x["volga0"] * ((x["iv"] - x["iv_lag"])**2), axis=1 )
df_pv_target["vanna PnL"] = df_pv_target.apply(lambda x: x["vanna0"] * (x["iv"] - x["iv_lag"]) *(x["S"] - x["S_lag"]), axis=1 )
df_pv_target["theta PnL"] = df_pv_target.apply(lambda x: x["theta0"] * x["t_passed"] , axis=1 )
df_pv_target["veta PnL"] = df_pv_target.apply(lambda x: x["veta0"] * (x["iv"] - x["iv_lag"]) * (x["t_passed"]), axis=1 )
df_pv_target["speed PnL"] = df_pv_target.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S"] - x["S_lag"]),3)), axis=1 )
df_pv_target["charm PnL"] = df_pv_target.apply(lambda x: x["charm0"] * (x["S"] - x["S_lag"]) * (x["t_passed"]), axis=1 )

#Marginal Changes
df_pv_target["BS_lag"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["BS"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S"], x["K"], x["r"], x["r_f"], x["iv"],  x["dt"]), axis=1 )

df_pv_target["BS0_ds"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt_lag"]), axis=1 )
df_pv_target["BS0_dsdv"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S"], x["K"], x["r"], x["r_f"], x["iv"],  x["dt_lag"]), axis=1 )
df_pv_target["BS0_dsdt"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt"]), axis=1 )

df_pv_target["BS0_dv"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv"],  x["dt_lag"]), axis=1 )
df_pv_target["BS0_dvds"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S"], x["K"], x["r"], x["r_f"], x["iv"],  x["dt_lag"]), axis=1 )
df_pv_target["BS0_dvdt"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv"],  x["dt"]), axis=1 )

df_pv_target["BS0_dt"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt"]), axis=1 )
df_pv_target["BS0_dtds"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S"], x["K"], x["r"], x["r_f"], x["iv_lag"],  x["dt"]), axis=1 )
df_pv_target["BS0_dtdv"] = df_pv_target.apply(lambda x: BSM_call_vec_with_div(x["S_lag"], x["K"], x["r"], x["r_f"], x["iv"],  x["dt"]), axis=1 )




In [16]:
print(df_pv_target)

            Date     Date_lag Month_Expiry   Week_Start  Week_Expiry  \
0    26-Jul-2019  25-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
1    29-Jul-2019  26-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
2    30-Jul-2019  29-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
3    31-Jul-2019  30-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
4    01-Aug-2019  31-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
..           ...          ...          ...          ...          ...   
186  17-Jul-2020  16-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
187  20-Jul-2020  17-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
188  21-Jul-2020  20-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
189  22-Jul-2020  21-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
190  23-Jul-2020  22-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   

            S     S_lag      K     r  q  ...     BS     BS0_ds   BS0_dsdv  \
0    11284.30  11252.15  12500  0.04  0  ...   2.75   3.21

In [17]:


# def BSM_call_with_div(s_t, k, r, r_f, vol, dt):
#     if (s_t == None):
#         return None
#     if (vol == None):
#         return None
#     else:
#         try:
#             d_1_c = (1 / (vol * (dt ** 0.5))) 
#             d1_log =  (np.log(s_t/k) + ((r + (vol ** 2) / 2) * dt))
#             d_1 = d_1_c * d1_log
#             d_2 = d_1 - vol * (dt ** 0.5)
#             q = r_f - r
#             price = (np.exp(-q * dt) * s_t * norm.cdf(d_1) - k * np.exp(-r_f * dt) * norm.cdf(d_2))
#         except:
#             price = None
#         return price

# def BSM_put_vec_with_div(s_t, k, r, r_f, vol, dt):
#     if (vol == None):
#         return None
#     else:
#         try:
#             d_1_c = (1 / (vol * (dt ** 0.5))) 
#             d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
#             d_1 = d_1_c * d1_log
#             d_2 = d_1 - vol * (dt ** 0.5)
#             q = r_f - r  
#             price = ((k * np.exp(-r_f*dt) * norm.cdf(-d_2)) - np.multiply(s_t, norm.cdf(-d_1)) * np.exp(-q * dt) )
#         except:
#             price = None
#         return price
    

# def np_sign(x):
#     if (x == None):
#         return 0
#     try:
#         y = np.sign(x)
#     except:
#         y = 0
#     return y

# def np_sub(x, y):
#     if (x == None):
#         return 0
#     try:
#         z = x - y
#     except:
#         z = 0
#     return z
    
def get_hdg_pfl_summary(begin_wk, end_wk, prv_day, nxt_day, prod_moneyness, r_f, S_lag, S):
    df_ce_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_CE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")
    df_pe_mv = pd.read_csv(input_data_path + "OPTIDX_NIFTY_PE_" + begin_wk.strftime("%d-%b-%Y") + "_TO_" + end_wk.strftime("%d-%b-%Y") +  ".csv")

    df_ce_mv["Date"] = df_ce_mv.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1)
    df_pe_mv["Date"] = df_pe_mv.apply(lambda x: x['Date'].split("-")[0] + "-" + \
                                  x['Date'].split("-")[1]  + "-"  + "20" + x['Date'].split("-")[2] \
                                  if (len(x['Date'].split("-")[2]) == 2) else x['Date'], axis=1)

    df_ce_mv0 = df_ce_mv[df_ce_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
    df_ce_mv0 = df_ce_mv0[["Strike Price", "Settle Price"]]
    df_ce_mv1 = df_ce_mv[df_ce_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
    df_ce_mv1 = df_ce_mv1[["Strike Price", "Settle Price"]]
    
    df_pe_mv0 = df_pe_mv[df_pe_mv["Date"] == (prv_day).strftime("%d-%b-%Y")]
    df_pe_mv0 = df_pe_mv0[["Strike Price", "Settle Price"]]
    df_pe_mv1 = df_pe_mv[df_pe_mv["Date"] == (nxt_day).strftime("%d-%b-%Y")]
    df_pe_mv1 = df_pe_mv1[["Strike Price", "Settle Price"]]

    df_ce_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(output_data_path + "B1A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-r_f * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-r_f * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)

    l2 = len(df_wts_opt.axes[0])
#     df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    
    df_wts = df_wts_opt
    l = len(df_wts.axes[0])
    S0 = [S_lag for i in range(0,l)]
    S1 = [S for i in range(0,l)]
    dt0= [((end_wk - prv_day).days)/365 for i in range(0,l)]
    dt1 = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
    r = [r_f for i in range(0,l)]
    r_f = [r_f for i in range(0,l)]
    q = [0 for i in range(0,l)]
    df_wts["S0"] = S0
    df_wts["S1"] = S1
    df_wts["dt0"] = dt0
    df_wts["dt1"] = dt1
    df_wts["r"] = r
    df_wts["r_f"] = r_f
    df_wts["q"] = q
    df_hdg_pfl = df_wts
    
   
    df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: brentqc(error_func, 0.00001, 100, x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), axis=1 )
    df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: brentqc(error_func, 0.00001, 50, x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), axis=1 )

    df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


    df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
    df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
    df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


    df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
    df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
    df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


    df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

    df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

    df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

    df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

    df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

    df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


    df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
    df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )


    df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

    df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
    df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
    df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
    df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
    df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
    df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
    df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5* x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
    df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
    df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
    df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
    df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
    df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

    hdge_cash0 = []
    hdge_cash1 = []

    hdg_MV0 = []
    hdge_MV1 = []
    hdge_BS0 = []
    hdge_BS1 = []

    hdge_BS0_ds = []
    hdge_BS0_dsdv = []
    hdge_BS0_dsdt = []

    hdge_BS0_dv = []
    hdge_BS0_dvds = []
    hdge_BS0_dvdt = []

    hdge_BS0_dt = []
    hdge_BS0_dtds = []
    hdge_BS0_dtdv = []

    hdge_delta0 = []
    hdge_gamma0 = []
    hdge_vega0 = []
    hdge_volga0 = []
    hdge_vanna0 = []
    hdge_theta0 = []
    hdge_veta0 = []
    hdge_speed0 = []
    hdge_charm0 = []

    hdgPflrealPnL = []
    hdgPflrealOptionsPnL = []
    hdgPflPnL = []
    hdge_deltaPnL = []
    hdge_gammaPnL = []
    hdge_vegaPnL = []
    hdge_volgaPnL = []
    hdge_vannaPnL = []
    hdge_thetaPnL = []
    hdge_vetaPnL = []
    hdge_speedPnL = []
    hdge_charmPnL = []



    df_hdg_sub_pfl = df_hdg_pfl
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
#     dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "MV0": hdg_MV0, "MV1": hdge_MV1, \
#                      "BS0":hdge_BS0, "BS1":hdge_BS1, \
#                     "BS0_ds":hdge_BS0_ds, "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
#                     "BS0_dv":hdge_BS0_dv, "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
#                     "BS0_dt":hdge_BS0_dt, "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
#                     "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
#                     "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
#                     "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
#                     "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
#                     "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
#                     "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
#                     "Hdge Pfl Realised PnL":hdgPflrealPnL}


#     df_summary_hdg =pd.DataFrame(dict_summary_hdg)
#     print(df_summary_hdg)
#     return df_summary_hdg

    hedge_list = [hdge_cash0, hdge_cash1, hdg_MV0, hdge_MV1, hdge_BS0, hdge_BS1, \
                   hdge_BS0_ds, hdge_BS0_dsdv, hdge_BS0_dsdt, \
                   hdge_BS0_dv, hdge_BS0_dvds, hdge_BS0_dvdt, \
                   hdge_BS0_dt, hdge_BS0_dtds, hdge_BS0_dtdv, \
                   hdge_delta0, hdge_gamma0, hdge_vega0, \
                   hdge_volga0, hdge_vanna0, hdge_theta0, \
                   hdge_veta0,  hdge_speed0, hdge_charm0, \
                   hdge_deltaPnL, hdge_gammaPnL, hdge_vegaPnL, \
                   hdge_volgaPnL, hdge_vannaPnL, hdge_thetaPnL, \
                   hdge_vetaPnL, hdge_speedPnL, hdge_charmPnL, \
                   hdgPflrealPnL]
    final_hedge_list = list(itertools.chain(*hedge_list))
    return final_hedge_list


    

In [7]:

# list2d = [[1,2,3], [4], [5,6]]

# print(x)

In [24]:
print(get_hdg_pfl_summary(date(2020,3,26), date(2020,4,1), date(2020,3,26), date(2020,3,27), "ATM", 0.04, 8641.45, 8660.25))


[[631.4457243365093], [631.5149276598371], [146.73393969720513], [120.82581637109678], [146.73393969720277], [120.8258163711026], [156.4599210763168], [139.4442376143473], [136.55585448927457], [129.57029386829294], [139.4442376143473], [110.9306997636404], [126.80621125205936], [136.55585448927457], [110.9306997636404], [0.5151598779620145], [0.00023173893742457194], [246.45922303770675], [182.56424223313704], [0.034668874720101266], [-7197.252949049046], [12567.567783906721], [2.1129920548170903e-08], [0.2916290744509917], [9.685005705685494], [0.04095290502166719], [-17.46030665462783], [0.30092847300572734], [0.14726973378112965], [-17.642401530438256], [-1.2329389955923291], [2.3400231904773264e-05], [0.015020894793639543], [-23.832023626275113]]


In [18]:
df_pnl = df_pv_target
hdge_column_names = ["Hedge cash_lag", "Hedge cash", "Hedge Close_lag", "Hedge Close", \
        "Hedge BS0", "Hedge BS1", \
        "Hedge BS0_ds", "Hedge BS0_dsdv", "Hedge BS0_dsdt", \
        "Hedge BS0_dv", "Hedge BS0_dvds", "Hedge BS0_dvdt", \
        "Hedge BS0_dt", "Hedge BS0_dtds", "Hedge BS0_dtdv", \
       "Hedge delta0", "Hedge gamma0", "Hedge vega 0", \
        "Hedge volga 0", "Hedge vanna0", "Hedge theta0", "Hedge veta0", \
        "Hedge speed0", "Hedge charm0", \
       "Hedge delta PnL", "Hedge gamma PnL", "Hedge vega PnL", \
        "Hedge volga PnL", "Hedge vanna PnL", "Hedge theta PnL", "Hedge veta PnL", \
        "Hedge speed PnL", "Hedge charm PnL", \
       "Hedge PnL"]
df_pnl[hdge_column_names] = df_pnl.apply(lambda x: get_hdg_pfl_summary(datetime.strptime(x['Week_Start'], "%d-%b-%Y").date(), \
                                                                  datetime.strptime(x['Week_Expiry'], "%d-%b-%Y").date(), \
                                                                  datetime.strptime(x['Date_lag'], "%d-%b-%Y").date(), \
                                                                  datetime.strptime(x['Date'], "%d-%b-%Y").date(), \
                                                                  prod_moneyness, x['r_f'], x['S_lag'], x['S']), axis=1, result_type = "expand")

# df_pnl1 = df_pnl[df_pnl["Date"] == "26-Jul-2019"]
# df_pnl1[hdge_column_names] = df_pnl1.apply(lambda x: get_hdg_pfl_summary(datetime.strptime(x['Week_Start'], "%d-%b-%Y").date(), \
#                                                                   datetime.strptime(x['Week_Expiry'], "%d-%b-%Y").date(), \
#                                                                   datetime.strptime(x['Date_lag'], "%d-%b-%Y").date(), \
#                                                                   datetime.strptime(x['Date'], "%d-%b-%Y").date(), \
#                                                                   prod_moneyness, x['r_f'], x['S_lag'], x['S']), axis=1, result_type = "expand")

# print(df_pnl1)
df_pnl.to_csv(out_PnL_path + "historical_pnL_" + prod_moneyness + ".csv")

In [33]:
print(df_pnl)

            Date     Date_lag Month_Expiry   Week_Start  Week_Expiry  \
0    26-Jul-2019  25-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
1    29-Jul-2019  26-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
2    30-Jul-2019  29-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
3    31-Jul-2019  30-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
4    01-Aug-2019  31-Jul-2019  29-Aug-2019  25-Jul-2019  01-Aug-2019   
..           ...          ...          ...          ...          ...   
186  17-Jul-2020  16-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
187  20-Jul-2020  17-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
188  21-Jul-2020  20-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
189  22-Jul-2020  21-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   
190  23-Jul-2020  22-Jul-2020  30-Jul-2020  16-Jul-2020  23-Jul-2020   

            S     S_lag      K     r  q  ...  Hedge delta PnL  \
0    11284.30  11252.15  11300  0.04  0  ...        15.606950   
1    

In [19]:
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)

In [20]:
print(mthly_expiries_list)

[datetime.date(2019, 8, 29), datetime.date(2019, 9, 26), datetime.date(2019, 10, 31), datetime.date(2019, 11, 28), datetime.date(2019, 12, 26), datetime.date(2020, 1, 30), datetime.date(2020, 2, 27), datetime.date(2020, 3, 26), datetime.date(2020, 4, 30), datetime.date(2020, 5, 28), datetime.date(2020, 6, 25), datetime.date(2020, 7, 30)]


In [204]:
###########################################################
# Target Option PnL Attribution - 7th April - NIFTY CALL ##
###########################################################

# summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
#                 "S0": [8083.8 for i in range(0,3)], "S1": [8792.2 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
# #                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
#                  "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
#                  "K": [8600, 8000, 9500], "MV0": [230.25, 526.15, 44.4], "MV1": [590.25, 1045.55, 150.55], \
#                   "dt0": [((date(2020,4,30) - date(2020, 4, 3)).days)/365 for i in range(0,3)], \
#                   "dt1": [((date(2020,4,30) - date(2020, 4, 7)).days)/365 for i in range(0,3)]}

# summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
#                 "S0": [8083.8 for i in range(0,3)], "S1": [8083.8 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
# #                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
#                  "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
#                  "K": [8600, 8000, 9500], "MV0": [230.25, 526.15, 44.4], "MV1": [590.25, 1045.55, 150.55], \
#                   "dt0": [((date(2020,4,30) - date(2020, 4, 3)).days)/365 for i in range(0,3)], \
#                   "dt1": [((date(2020,4,30) - date(2020, 4, 7)).days)/365 for i in range(0,3)]}

df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x[""], axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["delta1"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["vega1"] = df_summary.apply(lambda x: find_vega(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["gamma1"] = df_summary.apply(lambda x: find_gamma(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
# df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["vanna1"] = df_summary.apply(lambda x: find_vanna(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
# df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["volga1"] = df_summary.apply(lambda x: find_volga(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
# df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )


df_summary["Realised PnL"] = df_summary.apply(lambda x: x["MV1"] - x["MV0"], axis=1 )
df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp(1/365)) - \
                                                   x["MV0"], axis=1 )
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["S1"] - x["S0"]) * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"]  * ((x["dt0"] - x["dt1"])**2), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: 0.5 * x["speed0"] * ((x["S1"] - x["S0"])**3), axis=1 )


df_summary["Theoretical PnL"] = df_summary.apply(lambda x: x["delta PnL"] + x["gamma PnL"] + x["vega PnL"], axis=1 )
df_summary["All Risk PnL"] = df_summary.apply(lambda x: x["delta PnL"] + x["gamma PnL"] + x["vega PnL"] + \
                                              x["vanna PnL"] + x["volga PnL"], axis=1 )

df_summary.to_csv(out_PnL_path + "pnl_attribution_target_option_7April.csv")
print(df_summary)


  Option Type Moneyness      S0      S1   r_f     r    q     K     MV0  \
0          CE       ATM  8083.8  8083.8  0.04  0.04  0.0  8600  230.25   
1          CE       ITM  8083.8  8083.8  0.04  0.04  0.0  8000  526.15   
2          CE       OTM  8083.8  8083.8  0.04  0.04  0.0  9500   44.40   

       MV1  ...  Dynamic Hedge PnL  delta PnL  gamma PnL    vega PnL  \
0   590.25  ...          -7.109998        0.0        0.0  406.426873   
1  1045.55  ...         -11.094446        0.0        0.0  604.620451   
2   150.55  ...          -2.137508        0.0        0.0  100.934503   

   vanna PnL  volga PnL  theta PnL  speed PnL  Theoretical PnL  All Risk PnL  
0        0.0  42.616487  -0.328476        0.0       406.426873    449.043361  
1       -0.0   1.126445  -0.399083       -0.0       604.620451    605.746897  
2        0.0  52.542689  -0.142780        0.0       100.934503    153.477193  

[3 rows x 34 columns]


In [191]:
###########################################################
# Target Option PnL Attribution - 4th May - NIFTY CALL ##
###########################################################

summary_dict = {"Option Type": ["CE", "CE", "CE"], "Moneyness": ["ATM", "ITM", "OTM"], \
                "S0": [9859.9 for i in range(0,3)], "S1": [9293.5 for i in range(0,3)], "r_f": [0.04 for i in range(0,3)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [0.04 for i in range(0,3)], "q": [0.0 for i in range(0,3)], \
                 "K": [9900, 9000, 11000], "MV0": [280.65, 920.35, 16.75], "MV1": [116.8, 557.65, 9.15], \
                  "dt0": [((date(2020,5,28) - date(2020, 4, 30)).days)/365 for i in range(0,3)], \
                  "dt1": [((date(2020,5,28) - date(2020, 5, 4)).days)/365 for i in range(0,3)]}

df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV0"], x["S0"], x["K"], x["r"], x["r_f"], x["dt0"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: brentq(error_func, 0.00001, 50, args=(x["MV1"], x["S1"], x["K"], x["r"], x["r_f"], x["dt1"], x["Option Type"]), \
                                                      xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True), axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["delta1"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["vega1"] = df_summary.apply(lambda x: find_vega(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["gamma1"] = df_summary.apply(lambda x: find_gamma(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["vanna1"] = df_summary.apply(lambda x: find_vanna(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
# df_summary["volga1"] = df_summary.apply(lambda x: find_volga(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

df_summary["Realised PnL"] = df_summary.apply(lambda x: x["MV1"] - x["MV0"], axis=1 )
df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp(1/365)) - \
                                                   x["MV0"], axis=1 )
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["S1"] - x["S0"]) * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * ((x["dt0"] - x["dt1"])**2), axis=1 )

df_summary["Theoretical PnL"] = df_summary.apply(lambda x: x["delta PnL"] + x["gamma PnL"] + x["vega PnL"], axis=1 )
df_summary["All Risk PnL"] = df_summary.apply(lambda x: x["delta PnL"] + x["gamma PnL"] + x["vega PnL"] + \
                                              x["vanna PnL"] + x["volga PnL"] + x["theta PnL"], axis=1 )

df_summary.to_csv(out_PnL_path + "pnl_attribution_target_option_4MAY.csv")
print(df_summary)


  Option Type Moneyness      S0      S1   r_f     r    q      K     MV0  \
0          CE       ATM  9859.9  9293.5  0.04  0.04  0.0   9900  280.65   
1          CE       ITM  9859.9  9293.5  0.04  0.04  0.0   9000  920.35   
2          CE       OTM  9859.9  9293.5  0.04  0.04  0.0  11000   16.75   

      MV1  ...  Realised PnL  Dynamic Hedge PnL   delta PnL  gamma PnL  \
0  116.80  ...       -163.85        -301.313691 -288.314173  89.418505   
1  557.65  ...       -362.70        -535.203091 -513.217512  37.003612   
2    9.15  ...         -7.60         -35.736161  -34.151106  29.008429   

    vega PnL  vanna PnL  volga PnL  theta PnL  Theoretical PnL  All Risk PnL  
0  80.015081  -3.162658  -0.012678  -0.246133      -118.880586   -122.302055  
1  63.539601  61.608612  27.162847  -0.133775      -412.674300   -324.036616  
2  31.000526 -43.001280  15.276013  -0.064671        25.857849     -1.932088  

[3 rows x 32 columns]


In [165]:
# Static Hedge Pfl



323

In [197]:
#7th April - ATM
stock_vec = np.array([8083.8, 8792.2])
futures_price = np.array([8084.5, 8875.85])
dt = np.array([((date(2020,4,30) - date(2020, 4, 3)).days)/365, ((date(2020,4,30) - date(2020, 4, 7)).days)/365])
dt_short = np.array([((date(2020,4,9) - date(2020, 4, 3)).days)/365, ((date(2020,4,9) - date(2020, 4, 7)).days)/365])
r_f = np.array([0.04, 0.04])
r = np.log(np.divide(futures_price, stock_vec))/dt
q = r_f - r
opt_strike = 8600
mkt_price = np.array([230.25, 590.25])

i=0
iv_list_const_r = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i], "CE"), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list_const_r.append(iv)
iv_list_const_r = np.array(iv_list_const_r)
https://risklab.mgmt.iisc.ac.in/user/partha/notebooks/Vikranth%20-%20Chapter%202/Codes/F5_PnL_Attribution_NIFTY_CALL_7APR.ipynb#
i=0
delta = (BSM_call_vec_with_div(stock_vec[i]*1.01, opt_strike, r[i], r_f[i], iv_list_const_r[i], dt[i]) - BSM_call_vec_with_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list_const_r[i], dt[i]))/ (0.01*stock_vec[i])

vega = (BSM_call_vec_with_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list_const_r[i]*1.01, dt[i]) - BSM_call_vec_with_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list_const_r[i], dt[i]))/ (0.01*iv_list_const_r[i])

x_plus = BSM_call_vec_with_div(stock_vec[i]*1.01, opt_strike, r[i], r_f[i], iv_list_const_r[i], dt[i])
x = BSM_call_vec_with_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list_const_r[i], dt[i])
x_minus = BSM_call_vec_with_div(stock_vec[i] * 0.99, opt_strike, r[i], r_f[i], iv_list_const_r[i], dt[i])

gamma = (x_plus - 2* x + x_minus) / ((0.01 * stock_vec[i])**2)

print(delta, vega, gamma)

# iv_list = []
# for i in range(0,2):
#     iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[i], r_f[i], dt[i], "CE"), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
#     iv_list.append(iv)
# iv_list = np.array(iv_list)


# iv_list_const_r = []
# for i in range(0,2):
#     iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i], "CE"), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
#     iv_list_const_r.append(iv)
# iv_list_const_r = np.array(iv_list_const_r)


# delta_list = []
# for i in range(0,2):
#     delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     delta_list.append(delta)
# delta_list = np.array(delta_list)    


# delta_cir_list = []
# for i in range(0,2):
#     delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[0], r_f[0], iv_list[i], dt[i])
#     delta_cir_list.append(delta)
# delta_cir_list = np.array(delta_cir_list)    

# vega_list = []
# for i in range(0,2):
#     vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     vega_list.append(vega)
# vega_list = np.array(vega_list)

# vega_test_list = []
# for i in range(0,2):
#     vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     vega_test_list.append(vega)
# vega_test_list = np.array(vega_test_list)


# vanna_list = []
# for i in range(0,2):
#     vanna = find_vanna(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     vanna_list.append(vanna)
# vanna_list = np.array(vanna_list)

# volga_list = []
# for i in range(0,2):
#     volga = find_volga(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     volga_list.append(volga)
# volga_list = np.array(volga_list)

# gamma_list = []
# for i in range(0,2):
#     gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     gamma_list.append(gamma)
# gamma_list = np.array(gamma_list)

# gamma_test_list = []
# for i in range(0,2):
#     gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     gamma_test_list.append(gamma)
# gamma_test_list = np.array(gamma_list)


# theta_list = []
# for i in range(0,2):
#     theta = find_theta_Call(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
#     theta_list.append(theta)
# theta_list = np.array(theta_list)

# # print(delta_list)
# # print(vega_list)
# # print(iv_list)
# # print(iv_list_const_r)
# # print(delta_list)

# delta_hedge_pos = np.array([delta_list[0] * stock_vec[0], mkt_price[0] - delta_list[0]*stock_vec[1]])
# delta_PnL = delta_list[0] * (stock_vec[1] - stock_vec[0])
# delta_hedge_PnL = ((delta_hedge_pos[0] * stock_vec[1]) / stock_vec[0]) + delta_hedge_pos[1] * np.exp(r_f[0]/365) - np.sum(delta_hedge_pos)
# vega_constIR_PnL = vega_list[0] * (iv_list_const_r[1] - iv_list_const_r[0])
# gamma_PnL = gamma_list[0] * ((stock_vec[1] - stock_vec[0])**2) * 0.5
# vanna_cir_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list_const_r[1] - iv_list_const_r[0]))
# volga_cir_PnL = volga_list[0] * ((iv_list_const_r[1] - iv_list_const_r[0])**2)
# theta_PnL = theta_list[0] * (dt[0] - dt[1])

# realised_PnL = mkt_price[1] - mkt_price[0]

# # print(vega_list)
# # print(actual_PnL, delta_PnL, delta_hedge_PnL, vega_PnL, vega_constIR_PnL)

# target_opt_summary = {"Realised PnL": realised_PnL, "Delta PnL": delta_PnL, \
#                      "Delta Hedge PnL": delta_hedge_PnL, \
#                      "Vega PnL": vega_constIR_PnL, \
#                       "gamma PnL": gamma_PnL, \
#                      "Vanna PnL": vanna_cir_PnL, \
#                      "Volga PnL": volga_cir_PnL, \
#                      "Theta PnL": theta_PnL}
# print(target_opt_summary)

# print(vega_list)

0.3579109896902786 808.5717826182246 0.00034194690687607216


In [175]:
print(theta_list)
print(realised_PnL)
theoretical_PnL = delta_PnL + vega_constIR_PnL + gamma_PnL
print(theoretical_PnL)
print(delta_cir_list)


[-2660.69431076 -4176.91223048]
360.0
389.1898881922782
[0.34399757 0.59287421]


In [161]:
print(gamma_list)
print(gamma_test_list)

[0.0003421  0.00034432]
[0.0003421  0.00034432]


In [136]:
# ATM
df_hedge = pd.read_csv(input_data_path + "hedgePfl_7Apr.csv")

option_list = ["CE", "PE"]
date_list = np.unique(np.array(df_hedge["Date"]))

iv_ce_short_cir_list = []
iv_pe_short_cir_list = []

for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        mkt_price =  np.array(df["MV"])
        opt_strike = np.array(df["Strike"])
        iv_k_list = []
        for k in range(0, len_array):
            
#             print(option_type, opt_strike[k], mkt_price[k])
            iv = brentq(error_func_short, 0.0001, 10, args=(mkt_price[k], stock_vec[i], opt_strike[k], r[0], r_f[0], dt_short[i], option_type), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
#             iv_k_list.append(iv)
#             if (option_type == "CE"):
#                 model_price = BSM_call_vec_with_div(stock_vec[i], opt_strike[k], r[0], r_f[0], iv, dt_short[i])
#             else:
#                 model_price = BSM_put_vec_with_div(stock_vec[i], opt_strike[k], r[0], r_f[0], iv, dt_short[i])
#             print(model_price, mkt_price[k])
        if (j == 0):
            iv_ce_short_cir_list.append(iv_k_list)
        else:
            iv_pe_short_cir_list.append(iv_k_list)
    
            
          

delta_ce_short_list = []
delta_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        opt_strike = np.array(df["Strike"])
        delta_k_list = []
        for k in range(0, len_array):
            if (j == 0):
                delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_ce_short_list[i][k], dt_short[i])
            else:
                delta = put_delta_with_spot_and_div(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_pe_short_cir_list[i][k], dt_short[i])
            
            delta_k_list.append(delta)
    
        if (j == 0):
            delta_ce_short_list.append(delta_k_list)
        else:
            delta_pe_short_list.append(delta_k_list)
            

vega_ce_short_list = []
vega_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        opt_strike = np.array(df["Strike"])
        vega_k_list = []
        for k in range(0, len_array):
            vega = find_vega(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_ce_short_list[i][k], dt_short[i]) 
            vega_k_list.append(vega)
        if (j == 0):
            vega_ce_short_list.append(vega_k_list)
        else:
            vega_pe_short_list.append(vega_k_list)
                       

gamma_ce_short_list = []
gamma_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        opt_strike = np.array(df["Strike"])
        gamma_k_list = []
        for k in range(0, len_array):
            gamma = find_gamma(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_ce_short_list[i][k], dt_short[i]) 
            gamma_k_list.append(gamma)
        if (j == 0):
            gamma_ce_short_list.append(gamma_k_list)
        else:
            gamma_pe_short_list.append(gamma_k_list)                

            
df = df_hedge[df_hedge["Date"] == date_list[0]]
df1 = df_hedge[df_hedge["Date"] == date_list[1]]
ce_weights = np.array(df[df["Option"] == "CE"]["Weights"])
pe_weights = np.array(df[df["Option"] == "PE"]["Weights"])
hedge_pfl_delta_pnl = (np.dot(np.array(delta_ce_short_list[0]), ce_weights) + np.dot(np.array(delta_pe_short_list[0]), pe_weights)) * (stock_vec[1] - stock_vec[0])
# print(hedge_pfl_delta_pnl)


ce_iv_change_cir = np.array(iv_ce_short_cir_list[1]) - np.array(iv_ce_short_cir_list[0])
pe_iv_change_cir = np.array(iv_pe_short_cir_list[1]) - np.array(iv_pe_short_cir_list[0])

hedge_pfl_vega_cir_pnl = np.sum(np.multiply(np.multiply(np.array(vega_ce_short_list[0]), ce_weights), ce_iv_change_cir)) + \
                     np.sum(np.multiply(np.multiply(np.array(vega_pe_short_list[0]), pe_weights), pe_iv_change_cir))
hedge_pfl_gamma_pnl = 0.5 * (np.dot(np.array(gamma_ce_short_list[0]), ce_weights) + np.dot(np.array(gamma_pe_short_list[0]), pe_weights)) * ((stock_vec[1] - stock_vec[0])**2)

ce_mkt_list = [np.array(df[df["Option"] == "CE"]["MV"]), np.array(df1[df1["Option"] == "CE"]["MV"])]
pe_mkt_list = [np.array(df[df["Option"] == "PE"]["MV"]), np.array(df1[df1["Option"] == "PE"]["MV"])]
hdg_pfl_realised_PnL = (np.dot(ce_mkt_list[1], ce_weights)  + np.dot(pe_mkt_list[1], pe_weights)) - \
                       (np.dot(ce_mkt_list[0], ce_weights)  + np.dot(pe_mkt_list[0], pe_weights))


print(hdg_pfl_realised_PnL, hedge_pfl_delta_pnl, hedge_pfl_vega_pnl, hedge_pfl_vega_cir_pnl, hedge_pfl_gamma_pnl)




128.74999999999955 128.75
91.74999999999909 91.75
63.5 63.5
43.850000000000136 43.85
29.100000000001273 29.1
18.29999999999518 18.3
11.400000000000318 11.4
6.7000000000000455 6.7
4.950000000000188 4.95
3.0499999999998977 3.05
2.199999999999932 2.2
1.6000000000000085 1.6
1.3000000000001108 1.3
1.1999999999985391 1.2
1.1499999999966946 1.15
0.9999999999999574 1.0
0.799999999999983 0.8
0.4500000000000206 0.45
6.549999999999869 6.55
9.25 9.25
12.74999999999983 12.75
16.59999999999974 16.6
22.55000000000439 22.55
30.900000000018622 30.9
41.85000000000127 41.85
55.40000000000032 55.4
72.7999999999995 72.8
92.90000000000009 92.9
117.99999999999909 118.0
147.29999999999927 147.3
180.75 180.75
225.30000000000064 225.3
278.1000000000431 278.1
303.54999999999836 303.55
339.0 339.0
671.6000000000076 671.6
580.5499999999993 580.55
482.9499999999962 482.95
396.04999999999836 396.05
314.6000000000686 314.6
239.55000000001473 239.55
176.64999999999873 176.65
124.65000000000055 124.65
84.64999999999964

In [131]:
print(hdg_pfl_realised_PnL)
hdg_pfl_theoretical_PnL = hedge_pfl_delta_pnl + hedge_pfl_vega_cir_pnl + hedge_pfl_gamma_pnl
print(hdg_pfl_theoretical_PnL)

359.45918622280004
458.6402069468196


In [ ]:
def put_delta_with_spot(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    dc_ds = norm.cdf(d_1)
    # put delta = call delta - 1
    return (dc_ds-1)

def put_delta_with_spot_and_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    q = r_f - r 
    dc_ds = np.exp(- q * dt) * norm.cdf(d_1)
    return (dc_ds-np.exp(-q*dt))

# Function to obtain gama with respect to spot
def gamma_with_spot(s_t, k, r, vol, dt):
    d_1_g = (1 / (vol * (dt ** 0.5)))
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_g * d1_log
    gamma = norm.pdf(d_1) * (1 / (vol * s_t * dt)) 
    return gamma


{'30-Apr-2020': [datetime.date(2020, 3, 26), datetime.date(2020, 4, 1), datetime.date(2020, 4, 9), datetime.date(2020, 4, 16), datetime.date(2020, 4, 23)], '28-May-2020': [datetime.date(2020, 4, 30), datetime.date(2020, 5, 7), datetime.date(2020, 5, 14), datetime.date(2020, 5, 21)]}


In [118]:
#7th April - ITM
stock_vec = np.array([8083.8, 8792.2])
futures_price = np.array([8084.5, 8875.85])
dt = np.array([((date(2020,4,30) - date(2020, 4, 3)).days)/365, ((date(2020,4,30) - date(2020, 4, 7)).days)/365])
dt_short = np.array([((date(2020,4,9) - date(2020, 4, 3)).days)/365, ((date(2020,4,9) - date(2020, 4, 7)).days)/365])
r_f = np.array([0.04, 0.04])
r = np.log(np.divide(futures_price, stock_vec))/dt
q = r_f - r
opt_strike = 8000
mkt_price = np.array([526.15, 1045.55])



iv_list = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[i], r_f[i], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list.append(iv)
iv_list = np.array(iv_list)


iv_list_const_r = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list_const_r.append(iv)
iv_list_const_r = np.array(iv_list_const_r)


delta_list = []
for i in range(0,2):
    delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    delta_list.append(delta)
delta_list = np.array(delta_list)    


vega_list = []
for i in range(0,2):
    vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vega_list.append(vega)
vega_list = np.array(vega_list)

vanna_list = []
for i in range(0,2):
    vanna = find_vanna(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vanna_list.append(vanna)
vanna_list = np.array(vanna_list)

volga_list = []
for i in range(0,2):
    volga = find_volga(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    volga_list.append(volga)
volga_list = np.array(volga_list)

gamma_list = []
for i in range(0,2):
    gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    gamma_list.append(gamma)
gamma_list = np.array(gamma_list)

theta_list = []
for i in range(0,2):
    theta = find_theta_Call(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    theta_list.append(theta)
theta_list = np.array(theta_list)

# print(delta_list)
# print(vega_list)
# print(iv_list)
# print(iv_list_const_r)
# print(delta_list)

delta_hedge_pos = np.array([delta_list[0] * stock_vec[0], mkt_price[0] - delta_list[0]*stock_vec[1]])
delta_PnL = delta_list[0] * (stock_vec[1] - stock_vec[0])
delta_hedge_PnL = ((delta_hedge_pos[0] * stock_vec[1]) / stock_vec[0]) + delta_hedge_pos[1] * np.exp(r_f[0]/365) - np.sum(delta_hedge_pos)
vega_PnL = vega_list[0] * (iv_list[1] - iv_list[0])
vega_constIR_PnL = vega_list[0] * (iv_list_const_r[1] - iv_list_const_r[0])
gamma_PnL = gamma_list[0] * ((stock_vec[1] - stock_vec[0])**2) / 2
vanna_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list[1] - iv_list[0]))
vanna_cir_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list_const_r[1] - iv_list_const_r[0]))
volga_PnL = volga_list[0] * ((iv_list[1] - iv_list[0])**2)
volga_cir_PnL = volga_list[0] * ((iv_list_const_r[1] - iv_list_const_r[0])**2)
theta_PnL = theta_list[0] * (dt[1] - dt[0])

actual_PnL = mkt_price[1] - mkt_price[0]

# print(vega_list)
# print(actual_PnL, delta_PnL, delta_hedge_PnL, vega_PnL, vega_constIR_PnL)

target_opt_summary = {"Realised PnL": actual_PnL, "Delta PnL": delta_PnL, \
                     "Delta Hedge PnL": delta_hedge_PnL, "Vega PnL": vega_PnL, \
                     "Vega ConstIR PnL": vega_constIR_PnL, "gamma PnL": gamma_PnL, \
                     "Vanna PnL": vanna_PnL, "Vanna CIR PnL": vanna_cir_PnL, \
                     "Volga PnL": volga_PnL, "Volga CIR PnL": volga_cir_PnL, \
                     "Theta PnL": theta_PnL}
print(target_opt_summary)

# # Delta hedge
# stock_pos = delta*stock_vec
# deposit = mkt_price - delta*stock_vec




{'Realised PnL': 519.4, 'Delta PnL': 393.92662548235694, 'Delta Hedge PnL': 393.44846108652837, 'Vega PnL': 22.910122308486457, 'Vega ConstIR PnL': 102.44770655578284, 'gamma PnL': 80.86706709687205, 'Vanna PnL': 0.07974845652495668, 'Vanna CIR PnL': 0.3566129574663592, 'Volga PnL': -0.0009506221852992682, 'Volga CIR PnL': -0.019008927359186384, 'Theta PnL': 35.43972541322724}


In [119]:
393 + 102

495

In [120]:
#7th April - OTM
stock_vec = np.array([8083.8, 8792.2])
futures_price = np.array([8084.5, 8875.85])
dt = np.array([((date(2020,4,30) - date(2020, 4, 3)).days)/365, ((date(2020,4,30) - date(2020, 4, 7)).days)/365])
dt_short = np.array([((date(2020,4,9) - date(2020, 4, 3)).days)/365, ((date(2020,4,9) - date(2020, 4, 7)).days)/365])
r_f = np.array([0.04, 0.04])
r = np.log(np.divide(futures_price, stock_vec))/dt
q = r_f - r
opt_strike = 9500
mkt_price = np.array([44.4, 150.55])



iv_list = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[i], r_f[i], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list.append(iv)
iv_list = np.array(iv_list)


iv_list_const_r = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list_const_r.append(iv)
iv_list_const_r = np.array(iv_list_const_r)


delta_list = []
for i in range(0,2):
    delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    delta_list.append(delta)
delta_list = np.array(delta_list)    


vega_list = []
for i in range(0,2):
    vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vega_list.append(vega)
vega_list = np.array(vega_list)

vanna_list = []
for i in range(0,2):
    vanna = find_vanna(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vanna_list.append(vanna)
vanna_list = np.array(vanna_list)

volga_list = []
for i in range(0,2):
    volga = find_volga(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    volga_list.append(volga)
volga_list = np.array(volga_list)

gamma_list = []
for i in range(0,2):
    gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    gamma_list.append(gamma)
gamma_list = np.array(gamma_list)

theta_list = []
for i in range(0,2):
    theta = find_theta_Call(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    theta_list.append(theta)
theta_list = np.array(theta_list)

# print(delta_list)
# print(vega_list)
# print(iv_list)
# print(iv_list_const_r)
# print(delta_list)

delta_hedge_pos = np.array([delta_list[0] * stock_vec[0], mkt_price[0] - delta_list[0]*stock_vec[1]])
delta_PnL = delta_list[0] * (stock_vec[1] - stock_vec[0])
delta_hedge_PnL = ((delta_hedge_pos[0] * stock_vec[1]) / stock_vec[0]) + delta_hedge_pos[1] * np.exp(r_f[0]/365) - np.sum(delta_hedge_pos)
vega_PnL = vega_list[0] * (iv_list[1] - iv_list[0])
vega_constIR_PnL = vega_list[0] * (iv_list_const_r[1] - iv_list_const_r[0])
gamma_PnL = gamma_list[0] * ((stock_vec[1] - stock_vec[0])**2) / 2
vanna_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list[1] - iv_list[0]))
vanna_cir_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list_const_r[1] - iv_list_const_r[0]))
volga_PnL = volga_list[0] * ((iv_list[1] - iv_list[0])**2)
volga_cir_PnL = volga_list[0] * ((iv_list_const_r[1] - iv_list_const_r[0])**2)
theta_PnL = theta_list[0] * (dt[1] - dt[0])

actual_PnL = mkt_price[1] - mkt_price[0]

# print(vega_list)
# print(actual_PnL, delta_PnL, delta_hedge_PnL, vega_PnL, vega_constIR_PnL)

target_opt_summary = {"Realised PnL": actual_PnL, "Delta PnL": delta_PnL, \
                     "Delta Hedge PnL": delta_hedge_PnL, "Vega PnL": vega_PnL, \
                     "Vega ConstIR PnL": vega_constIR_PnL, "gamma PnL": gamma_PnL, \
                     "Vanna PnL": vanna_PnL, "Vanna CIR PnL": vanna_cir_PnL, \
                     "Volga PnL": volga_PnL, "Volga CIR PnL": volga_cir_PnL, \
                     "Theta PnL": theta_PnL}
print(target_opt_summary)

# # Delta hedge
# stock_pos = delta*stock_vec
# deposit = mkt_price - delta*stock_vec




{'Realised PnL': 106.15, 'Delta PnL': 71.35090516012212, 'Delta Hedge PnL': 71.25871807900296, 'Vega PnL': -9.669603662056714, 'Vega ConstIR PnL': 2.1853671507241925, 'gamma PnL': 45.314885657870185, 'Vanna PnL': -9.800502067700691, 'Vanna CIR PnL': 2.2149506875240528, 'Volga PnL': 0.9681721049361801, 'Volga CIR PnL': 0.04945202040984683, 'Theta PnL': 12.737096391642162}


In [116]:
def find_veta(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    veta = -s_t * np.exp(-q*dt) * norm.pdf(d_1) * np.sqrt(dt) * \
    (q + ((r_f-q) * d_1) / (vol * np.sqrt(dt)) - (1 + d_1 * d_2) / (2 * dt) )
    return veta

print(find_veta(100, 100, 0.04, 0.04, 0.5, 0.1))


print((BSM_call_vec_with_div(100, 100, 0.04, 0.04, 0.49, 0.09) +  BSM_call_vec_with_div(100, 100, 0.04, 0.04, 0.51, 0.11) \
      - BSM_call_vec_with_div(100, 100, 0.04, 0.04, 0.49, 0.11)  \
       - BSM_call_vec_with_div(100, 100, 0.04, 0.04, 0.51, 0.09)) / (4* np.square(0.01)))
      


62.05259231598302
62.132121674665086


In [40]:
np.sign(-2) * np.sign(3)

-1

In [32]:
vol = None
if(vol == None):
    print("Y")

Y
